In [ ]:
# Load Giotto, data.table is also necessary for this example
library(Giotto)
#library(GiottoData)
library(data.table)
# add color palettes if you want!
#library(rcartocolor)

In [ ]:
library(ggplot2)

# Create data
data <- rnorm(1000)

In [ ]:
data <- as.data.frame(data)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
# Plot histogram with ggplot2
ggplot(data = data, aes(x = data)) +
  geom_histogram(fill = "blue", color = "black", alpha = 0.5) +
  labs(x = "Data", y = "Count") +
  theme_classic()

In [ ]:
dim(pDataDT(fov_join))

In [ ]:
counts <- get_expression_values(fov_join)@exprMat

In [ ]:
neg_counts <- (counts[grepl("^Neg",rownames(counts)),])

In [ ]:
nonneg_counts <- (counts[!grepl("^Neg",rownames(counts)),])

In [ ]:
dim(nonneg_counts)

In [ ]:
dim(neg_counts)

In [ ]:
neg_sum <- apply(neg_counts,2,sum)

In [ ]:
all_sum <- apply(counts,2,sum)

In [ ]:
head(nonneg_sum)

In [ ]:
neg_sum <- as.data.frame(neg_sum)
all_sum <- as.data.frame(all_sum)

In [ ]:
data <- cbind(neg_sum,all_sum)

In [ ]:
colnames(data) <- c('neg_probe', 'RNA')

In [ ]:
library(dplyr)

In [ ]:
head(data)

In [ ]:
head(data)

In [ ]:
options(repr.plot.width=5,repr.plot.height=3)
ggplot(data, aes(x=RNA, fill="RNA")) +
  geom_histogram(bins=100, alpha=0.5, fill = 'blue', color='black', size=0.3, show.legend = TRUE) +
  geom_histogram(aes(x=neg_probe, fill="neg_probe"), fill = 'red', alpha=1, bins=150) +
  scale_fill_brewer()+
  theme_classic() +
  scale_y_log10() +
  ggtitle("Histograms")

In [ ]:
options(repr.plot.width=5,repr.plot.height=3)
ggplot(data, aes(x=RNA, fill="RNA")) +
  geom_histogram(bins=100, alpha=0.5, fill = 'blue', color='black', size=0.3, show.legend = TRUE) +
  geom_histogram(aes(x=neg_probe, fill="neg_probe"), fill = 'red', alpha=1, bins=150) +
  scale_fill_brewer()+
  theme_classic() +
  scale_y_log10() +
  ggtitle("Histograms")

In [ ]:
options(repr.plot.width=3,repr.plot.height=3)
ggplot(data, aes(x=neg_probe, fill="neg_probe", color='neg_probe')) +
  geom_histogram(bins=70, alpha=0.3, fill = 'red',color='white', size=0.01) +
  geom_histogram(aes(x=RNA, fill="RNA",color='RNA'), fill = 'blue', color='white',size=0.1, alpha=0.3, bins=70) +
  scale_fill_brewer()+
  theme_classic() +
  scale_y_log10() +
  labs(x = "total features per cell", y = "log10 count")

In [ ]:
# Plot histograms
options(repr.plot.width=3,repr.plot.height=3)
ggplot() +
  geom_histogram(data = data, aes(x = RNA, fill=RNA), fill = "blue", color='black', alpha = 0.5, bins=100) +
  geom_histogram(data = data, aes(x = neg_probe, fill=neg_probe), fill = "red", alpha = 1,bins=150) +
  labs(x = "Data", y = "Count") +
  scale_fill_discrete(name="") +
  guides(fill=guide_legend(keywidth=1, keyheight=1))+
  theme_classic()

In [ ]:
packageVersion('Giotto')

In [ ]:
library(terra)

In [ ]:
results_folder = '/home/tsubosaka/cosmx_results/'

In [ ]:
1+1

In [ ]:
# set giotto python path
# set python path to your preferred python version path
# set python path to NULL if you want to automatically install (only the 1st time) and use the giotto miniconda environment
python_path = NULL
#if(is.null(python_path)) {
 # installGiottoEnvironment(force_environment = TRUE)
#}


## create instructions
# by directly saving plots, but not rendering them you will save a lot of time
instrs = createGiottoInstructions(save_dir = results_folder, dpi = 350,
                                  save_plot = TRUE,
                                  show_plot = TRUE,
                                  return_plot = FALSE,
                                 plot_format = 'png')

# Data load1

## Choose field of view for analysis
## before ver 3.0

In [ ]:
## provide path to nanostring folder
data_path = '/home/tsubosaka/5006/'

# load transcript coordinates
tx_coord_all = fread(paste0(data_path, 'Run5458_5006_tx_file.csv'))

#  load field of vision (fov) positions
fov_offset_file = fread(paste0(data_path, 'Run5458_5006_fov_positions_file.csv'))

In [ ]:
gobjects_list = list()

# select which FOV's you would like to work with
# the dataset includes 28, which is too much for most computers to handle at once.
#For this example I am using 02, 03, and 04
id_set = c('01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24')

##### 3. Create a Giotto Object for each FOV

In [ ]:
for(fov_i in 1:length(id_set)) {

  fov_id = id_set[fov_i]


  # 1. original composite image as png
  original_composite_image = paste0(data_path, 'CellComposite/CellComposite_F0', fov_id,'.jpg')

  # 2. input cell segmentation as mask file
  segmentation_mask = paste0(data_path, 'CellLabels/CellLabels_F0', fov_id, '.tif')

  # 3. input features coordinates + offset
  tx_coord = tx_coord_all[fov == as.numeric(fov_id)]
  tx_coord = tx_coord[,.(x_local_px, y_local_px, z, target)]
  colnames(tx_coord) = c('x', 'y', 'z', 'gene_id')
  tx_coord = tx_coord[,.(x, y, gene_id)]


  fovsubset = createGiottoObjectSubcellular(gpoints = list('rna' = tx_coord),
                                            gpolygons = list('cell' = segmentation_mask),
                                            polygon_mask_list_params = list(mask_method = 'guess',
                                                                            flip_vertical = TRUE,
                                                                            flip_horizontal = FALSE,
                                                                            shift_horizontal_step = FALSE),
                                            instructions = instrs)


  # centroids are now used to provide the spatial locations (centroid of each cell)
  fovsubset = addSpatialCentroidLocations(fovsubset,
                                          poly_info = 'cell')

  # create and add Giotto images
  composite = createGiottoLargeImage(raster_object = original_composite_image,
                                     negative_y = FALSE,
                                     name = 'composite')

  fovsubset = addGiottoImage(gobject = fovsubset,
                             largeImages = list(composite))


  fovsubset = convertGiottoLargeImageToMG(giottoLargeImage = composite,
                                          #mg_name = 'composite',
                                          gobject = fovsubset,
                                          return_gobject = TRUE)

  gobjects_list[[fov_i]] = fovsubset


}

##### 4. Join Giotto Objects

In [ ]:
new_names = paste0("fov0", id_set)

id_match = match(as.numeric(id_set), fov_offset_file$fov)
x_shifts = fov_offset_file[id_match]$x_global_px
y_shifts = fov_offset_file[id_match]$y_global_px

# Create Giotto object that includes all selected FOVs
fov_join = joinGiottoObjects(gobject_list = gobjects_list,
                             gobject_names = new_names,
                             join_method = 'shift',
                             x_shift = x_shifts,
                             y_shift = y_shifts)

In [ ]:
showGiottoImageNames(fov_join)
options(repr.plot.width=21, repr.plot.height=21)
# Set up vector of image names
id_set = c('1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24')
new_names = paste0("fov0", id_set)
image_names = paste0(new_names, '-image')

##### 2'-4' additional data

In [ ]:
## provide path to nanostring folder
data_path2 = '/home/tsubosaka/0507/'

# load transcript coordinates
tx_coord_all2 = fread(paste0(data_path2, 'Run5458_0507_tx_file.csv'))

#  load field of vision (fov) positions
fov_offset_file2 = fread(paste0(data_path2, 'Run5458_0507_fov_positions_file.csv'))

In [ ]:
gobjects_list2 = list()

# select which FOV's you would like to work with
# the dataset includes 28, which is too much for most computers to handle at once.
#For this example I am using 02, 03, and 04
id_set2 = c('01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23')

In [ ]:
for(fov_i in 1:length(id_set2)) {

  fov_id2 = id_set2[fov_i]


  # 1. original composite image as png
  original_composite_image = paste0(data_path2, 'CellComposite/CellComposite_F0', fov_id2,'.jpg')

  # 2. input cell segmentation as mask file
  segmentation_mask = paste0(data_path2, 'CellLabels/CellLabels_F0', fov_id2, '.tif')

  # 3. input features coordinates + offset
  tx_coord = tx_coord_all[fov == as.numeric(fov_id2)]
  tx_coord = tx_coord[,.(x_local_px, y_local_px, z, target)]
  colnames(tx_coord) = c('x', 'y', 'z', 'gene_id')
  tx_coord = tx_coord[,.(x, y, gene_id)]


  fovsubset = createGiottoObjectSubcellular(gpoints = list('rna' = tx_coord),cores = 2,
                                            gpolygons = list('cell' = segmentation_mask),
                                            polygon_mask_list_params = list(mask_method = 'guess',
                                                                            flip_vertical = TRUE,
                                                                            flip_horizontal = FALSE,
                                                                            shift_horizontal_step = FALSE),
                                            instructions = instrs)


  # centroids are now used to provide the spatial locations (centroid of each cell)
  fovsubset = addSpatialCentroidLocations(fovsubset,
                                          poly_info = 'cell')

  # create and add Giotto images
  composite = createGiottoLargeImage(raster_object = original_composite_image,
                                     negative_y = FALSE,
                                     name = 'composite')

  fovsubset = addGiottoImage(gobject = fovsubset,
                             largeImages = list(composite))


  fovsubset = convertGiottoLargeImageToMG(giottoLargeImage = composite,
                                          #mg_name = 'composite',
                                          gobject = fovsubset,
                                          return_gobject = TRUE)

  gobjects_list2[[fov_i]] = fovsubset


}

In [ ]:
new_names2 = paste0("fov0", id_set2)

id_match2 = match(as.numeric(id_set2), fov_offset_file2$fov)
x_shifts2 = fov_offset_file2[id_match2]$x_global_px
y_shifts2 = fov_offset_file2[id_match2]$y_global_px

In [ ]:
# Create Giotto object that includes all selected FOVs
fov_join2 = joinGiottoObjects(gobject_list = gobjects_list2,
                             gobject_names = new_names2,
                             join_method = 'shift',
                             x_shift = x_shifts2,
                             y_shift = y_shifts2)

In [ ]:
showGiottoImageNames(fov_join2)
options(repr.plot.width=21, repr.plot.height=21)
# Set up vector of image names
id_set2 = c('1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23')
new_names2 = paste0("fov0", id_set2)
image_names2 = paste0(new_names2, '-image')

# Data load new 
## after ver 3.0

In [ ]:
# Fundic gland
data_path = '/data/share/spatial//CosMX//normal_stomach//tsubosaka//FinalAnalysis//Run5458_5006/'

In [ ]:
data_path

In [ ]:
fov_join <- createGiottoCosMxObject(cosmx_dir = data_path,data_to_use = 'subcellular', instructions = instrs, cores = 4)

In [ ]:
# Pyloric gland
data_path2 = '/data/share/spatial/CosMX/normal_stomach/tsubosaka/FinalAnalysis/Run5458_0507/'

In [ ]:
fov_join2 <- createGiottoCosMxObject(cosmx_dir = data_path2,data_to_use = 'subcellular',instructions = instrs, cores = 4)

In [ ]:
fov_join@feat_info

# join

fov_join = loadGiotto('./saveGiottoDir_final_before_submission/')

In [ ]:
# integration
fov_join = joinGiottoObjects(gobject_list = list(fov_join2, fov_join),
                                gobject_names = c('Pyloric','Fundic'),
                                join_method='no_change'
                                )

In [ ]:
fov_join = calculateOverlapRaster(fov_join)

fov_join = overlapToMatrix(fov_join)

showGiottoExpression(fov_join)

# combine cell data
morphometa = combineCellData(fov_join,
                             feat_type = 'rna')

# combine feature data
featmeta = combineFeatureData(fov_join,
                              feat_type = c('rna'))

# combine overlapping feature data
featoverlapmeta = combineFeatureOverlapData(fov_join,
                                            feat_type = c('rna'))

In [ ]:
fov_join@feat_info$rna

In [ ]:
1+1

In [ ]:
fov_join = calculateOverlapRaster(fov_join, feat_info = 'neg_probe')

In [ ]:
# Find the feature points overlapped by polygons. This overlap information is then
# returned to the relevant giottoPolygon object's overlaps slot.

# Convert the overlap information into a cell by feature expression matrix which
# is then stored in the Giotto object's expression slot
fov_join = overlapToMatrix(fov_join, feat_info = 'rna')
fov_join = overlapToMatrix(fov_join, feat_info = 'neg_probe')

In [ ]:
showGiottoExpression(fov_join)

In [ ]:
options(repr.plot.width=5,repr.plot.height=3)
p1 <- filterDistributions(fov_join,
                    plot_type = 'hist',
                    detection = 'cells',
                    method = 'sum',
                    feat_type = 'rna',
                    nr_bins = 100,
                    save_param = list(base_height = 3,
                                      save_name = '3.1_totalexpr'))

p2 <- filterDistributions(fov_join,return_plot = TRUE,
                    plot_type = 'hist',
                    detection = 'cells',
                    method = 'sum',
                    feat_type = 'neg_probe',
                    nr_bins = 25,
                    save_param = list(base_height = 3,
                                      save_name = '3.2_totalnegprbe'))

In [ ]:
table(pDataDT(fov_join)$site)

In [ ]:
table(pDataDT(fov_join)$list_ID)

### add metadata (immunofluorscence and cell size, cell asepct ratio data)

In [ ]:
data_path = '/home/tsubosaka/5006/'

In [ ]:
#  load field of vision (fov) positions
metadata_file_5006 = fread(paste0(data_path, 'Run5458_5006_metadata_file.csv'))

In [ ]:
data_path2 = '/home/tsubosaka/0507/'

In [ ]:
#  load field of vision (fov) positions
metadata_file_0507 = fread(paste0(data_path2, 'Run5458_0507_metadata_file.csv'))

In [ ]:
metadata_file_5006 = as.data.frame(metadata_file_5006)
metadata_file_0507 = as.data.frame(metadata_file_0507)

In [ ]:
metadata_file_5006['site'] = 'Fundic'
metadata_file_0507['site'] = 'Pyloric'

In [ ]:
#met <- rbind(metadata_file_5006,metadata_file_0507)

In [ ]:
met <- rbind(metadata_file_0507,metadata_file_5006)

In [ ]:
1+1

In [ ]:
dim(met)

In [ ]:
met <- as.data.frame(met)

In [ ]:
met[met$fov==1,]$fov = '001'
met[met$fov==2,]$fov = '002'
met[met$fov==3,]$fov = '003'
met[met$fov==4,]$fov = '004'
met[met$fov==5,]$fov = '005'
met[met$fov==6,]$fov = '006'
met[met$fov==7,]$fov = '007'
met[met$fov==8,]$fov = '008'
met[met$fov==9,]$fov = '009'
met[met$fov==10,]$fov = '010'
met[met$fov==11,]$fov = '011'
met[met$fov==12,]$fov = '012'
met[met$fov==13,]$fov = '013'
met[met$fov==14,]$fov = '014'
met[met$fov==15,]$fov = '015'
met[met$fov==16,]$fov = '016'
met[met$fov==17,]$fov = '017'
met[met$fov==18,]$fov = '018'
met[met$fov==19,]$fov = '019'
met[met$fov==20,]$fov = '020'
met[met$fov==21,]$fov = '021'
met[met$fov==22,]$fov = '022'
met[met$fov==23,]$fov = '023'
met[met$fov==24,]$fov = '024'

In [ ]:
# take a litte time
fov_cellID <- c()
for (i in 1:268326){
    a <- paste0(met$site[i],'-fov',met$fov[i],'-cell_',met$cell_ID[i])
    fov_cellID <- c(fov_cellID, a)
    }

In [ ]:
length(fov_cellID)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = met)

In [ ]:
colnames(pDataDT(fov_join))

In [ ]:
table(pDataDT(fov_join)$list_ID)

In [ ]:
table(pDataDT(fov_join)$Slide_name)

In [ ]:
table(pDataDT(fov_join)$Run_name)

In [ ]:
meta <- pDataDT(fov_join)

In [ ]:
dim(fov_join@expression$cell$rna$raw)

#### subtract negative value

In [ ]:
counts_all <- Giotto:::get_expression_values(fov_join, feat_type = 'rna')

In [ ]:
counts_all <- counts_all@exprMat

In [ ]:
dim(counts_all)

In [ ]:
# ** Note: `counts_all[]` is shorthand for `counts_all@exprMat` **

# extract the raw matrix
counts_feat <- counts_all[rownames(counts_all)[!grepl('^Neg',rownames(counts_all ))],]
counts_neg <- counts_all[rownames(counts_all)[grepl('^Neg',rownames(counts_all ))],]

In [ ]:
table(colSums(as.matrix(counts_all))>=20)

In [ ]:
head(colnames(counts_feat))

In [ ]:
table(colSums(as.matrix(counts_feat))>=20)

In [ ]:
cell_id_use1 <- colnames(counts_feat)[colSums(as.matrix(counts_feat))>=20]

In [ ]:
cell_id_use2 <- pDataDT(fov_join)[pDataDT(fov_join)$Area <= 5*mean(pDataDT(fov_join)$Area)]$cell_ID

In [ ]:
head(cell_id_use1)

In [ ]:
length(cell_id_use1)

In [ ]:
head(cell_id_use2)

In [ ]:
length(cell_id_use2)

In [ ]:
cell_id_use <- intersect(cell_id_use1, cell_id_use2)

In [ ]:
length(cell_id_use)

In [ ]:
# calculate the mean of negative probe per cell
negmean <- Matrix::colMeans(counts_neg[])

In [ ]:
# subtract negmean
count_replace <- t(apply(counts_feat[],1,function(row) row - negmean))
count_replace[count_replace < 0 ] <- 0
count_replace <- ceiling(count_replace)

In [ ]:
# replace the matrix
counts_feat[] <- as(count_replace, 'dgCMatrix')
fov_join <- Giotto:::set_expression_values(fov_join, values = counts_feat)

In [ ]:
fov_join <- subsetGiotto(fov_join, cell_ids = cell_id_use)

In [ ]:
aaa <- readRDS('seurat_object.Rds')

In [ ]:
aaa

In [ ]:
library(Seurat)

In [ ]:
unique(aaa@meta.data$fov)

In [ ]:
metadata <- aaa@meta.data

In [ ]:
library(stringr)

In [ ]:
# replace the fov_join 
# take some time 
test10 <- c()
for (i in 1:dim(fov_join@cell_metadata$cell$rna )[1]){
    test <- fov_join@cell_metadata$cell$rna$cell_ID[i]
        if (str_detect(test, 'Fundic')) {
            test2 <- str_replace(test, 'Fundic-','Run5458.5006_')
            test3 <- str_split(test2, '_')[[1]]
            test4 <- paste0(test3[1], '_',test3[3],'_',test3[2])
            j <- as.integer(str_sub(test3[2],4,6))
#            print(test3[2])
 #           print(test4)
            
        if ( j < 10) {
        test4 <- str_replace(test4, paste0('fov00',j,'-cell'), as.character(j))
            } else {
            test4 <- str_replace(test4, paste0('fov0',j,'-cell'), as.character(j))
            }    
        } else {
            test2 <- str_replace(test, 'Pyloric-','Run5458.0507_')
            test3 <- str_split(test2, '_')[[1]]
            test4 <- paste0(test3[1], '_',test3[3],'_',test3[2])
            j <- as.integer(str_sub(test3[2],4,6))
#            print(test3[2])
 #           print(test4)
            
        if ( j < 10) {
        test4 <- str_replace(test4, paste0('fov00',j,'-cell'), as.character(j))
            } else {
            test4 <- str_replace(test4, paste0('fov0',j,'-cell'), as.character(j))
            }    
        }
            
            
    test10 <- c(test10,test4)
    }

In [ ]:
length(test10)

In [ ]:
head(test10)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = test10, vector_name = 'cell_ID_2')

In [ ]:
giotto_meta <- pDataDT(fov_join)

In [ ]:
colnames(giotto_meta)

In [ ]:
giotto_meta <- as.data.frame(giotto_meta)

In [ ]:
rownames(giotto_meta) = giotto_meta$cell_ID_2

In [ ]:
dim(giotto_meta)

In [ ]:
dim(metadata)

In [ ]:
head(rownames(metadata))

In [ ]:
head(rownames(giotto_meta))

In [ ]:
inter <- intersect(rownames(metadata), rownames(giotto_meta))

In [ ]:
length(inter)

In [ ]:
giotto_meta <- giotto_meta[inter,]

In [ ]:
head(giotto_meta)

In [ ]:
subset_cell_ids = giotto_meta$cell_ID

In [ ]:
head(subset_cell_ids)

In [ ]:
fov_join2 <- subsetGiotto(fov_join, cell_ids = subset_cell_ids)

In [ ]:
table(pDataDT(fov_join2)$list_ID)

In [ ]:
table(pDataDT(fov_join2)$site)

#### For figure and table

In [ ]:
options(repr.plot.width=4,repr.plot.height=3)
p1 <- filterDistributions(fov_join2,
                    plot_type = 'hist',
                    detection = 'cells',
                    method = 'sum',
                    feat_type = 'rna',
                    nr_bins = 100,
                    save_param = list(base_height = 3,
                                      save_name = '3.1_totalexpr'))

p2 <- filterDistributions(fov_join2,return_plot = TRUE,
                    plot_type = 'hist',
                    detection = 'cells',
                    method = 'sum',
                    feat_type = 'neg_probe',
                    nr_bins = 25,
                    save_param = list(base_height = 3,
                                      save_name = '3.2_totalnegprbe'))

In [ ]:
colnames(pDataDT(fov_join2))

In [ ]:
sum(pDataDT(fov_join2)[pDataDT(fov_join2)$list_ID=='Pyloric']$nCount_Nanostring)

In [ ]:
max(pDataDT(fov_join2)[pDataDT(fov_join2)$tissue=='Stomach_0507']$nCount_Nanostring)

In [ ]:
max(pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_0507']$Area)*0.324

In [ ]:
a <- pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_0507']

In [ ]:
a[a$list_ID=='Fundic']

In [ ]:
mean(pDataDT(fov_join)[pDataDT(fov_join)$list_ID=='Pyloric']$Area)*0.18*0.18

In [ ]:
mean(pDataDT(fov_join)[pDataDT(fov_join)$list_ID=='Fundic']$Area)*0.18*0.18

In [ ]:
counts <- get_expression_values(fov_join2)

In [ ]:
counts <- counts@exprMat

In [ ]:
head(counts)

In [ ]:
head(colnames(counts))

In [ ]:
counts_pyloric <- counts[,grepl('Pyloric',(colnames(counts)))]

In [ ]:
dim(counts_pyloric)

In [ ]:
max(apply(counts_pyloric,2,sum))

In [ ]:
sum(counts_pyloric)

In [ ]:
21932028/129818

In [ ]:
counts_fundic <- counts[,grepl('Fundic',(colnames(counts)))]

In [ ]:
dim(counts_fundic)

In [ ]:
sum(counts_fundic)

In [ ]:
max(apply(counts_fundic,2,sum))

In [ ]:
16598878/114627

#### return to analysis

In [ ]:
fov_join <- fov_join2

In [ ]:
remove(fov_join2)

In [ ]:
# normalize
# standard method
fov_join <- normalizeGiotto(gobject = fov_join,
                            scalefactor = 5000,
                            verbose = T)


In [ ]:
# new normalizaton method based on pearson correlations (Lause/Kobak et al. 2021)
# this normalized matrix is given the name 'pearson' and will be used in the downstream steps
fov_join <- normalizeGiotto(gobject = fov_join,
                            scalefactor = 5000,
                            verbose = T,
                            norm_methods = 'pearson_resid',
                            update_slot = 'pearson')

In [ ]:
colnames(fov_join@cell_metadata$cell$rna@metaDT)[4] <- 'cell_ID2'

In [ ]:
# add statistics
fov_join <- addStatistics(gobject = fov_join,)

# View cellular data
#pDataDT(fov_join)
# View rna data
#fDataDT(fov_join)

In [ ]:
1+1

In [ ]:
dim(fov_join@expression$cell$rna$raw)

# For InSituTyping

#### 不要

In [ ]:
aaa <- readRDS('seurat_object.Rds')

In [ ]:
aaa

In [ ]:
library(Seurat)

In [ ]:
unique(aaa@meta.data$fov)

In [ ]:
metadata <- aaa@meta.data

In [ ]:
library(stringr)

In [ ]:
tail(rownames(metadata))

In [ ]:
# replace the fov_join 
# take some time 
test10 <- c()
for (i in 1:dim(metadata)[1]){
    test <- rownames(metadata)[i]
        if (str_detect(test, 'Run5458.5006_')) {
            test2 <- str_replace(test,'Run5458.5006_','Fundic_')
            test3 <- str_split(test2, '_')[[1]]
            if (as.integer(test3[3]) < 10) {
            test4 <- paste0(test3[1], '-fov00',test3[3],'-cell_',test3[2])
                }else{
            test4 <- paste0(test3[1], '-fov0',test3[3],'-cell_',test3[2])
            }    
        } else {
            test2 <- str_replace(test, 'Run5458.0507_','Pyloric_')
            test3 <- str_split(test2, '_')[[1]]
            if (as.integer(test3[3]) < 10) {
            test4 <- paste0(test3[1], '-fov00',test3[3],'-cell_',test3[2])
                }else{
            test4 <- paste0(test3[1], '-fov0',test3[3],'-cell_',test3[2])
            }
        
        }   
    test10 <- c(test10,test4)
    }

In [ ]:
head(test10)

In [ ]:
length(test10)

In [ ]:
length(pDataDT(fov_join)$cell_ID)

In [ ]:
head(setdiff(test10, pDataDT(fov_join)$cell_ID))

In [ ]:
length(setdiff(pDataDT(fov_join)$cell_ID,test10))

In [ ]:
fov_join3 <- subsetGiotto(fov_join, cell_ids = test10)

#### produce reference matrix

In [ ]:
library(InSituType)

In [ ]:
data("ioprofiles")
data("iocolors")

In [ ]:
iocolors

In [ ]:
epi <- readRDS('/home/tsubosaka/for_analysis/論文用再検討/修正/data_9_99_monocle3_seurat_epi.rds')

In [ ]:
fb <- readRDS('/home/tsubosaka//for_analysis/論文用再検討/修正/data_9_3_annotated_fibroblasts.rds')

aaa <- readRDS('seurat_object.Rds')

In [ ]:
library(Seurat)

In [ ]:
table(epi@meta.data$subcluster)

#### B cell, endothelialまでやると細かすぎる可能性
B <- readRDS('./for_analysis/論文用再検討/修正/fig作成用/2022_data_9_9_annotated_seurat_all.rds')

B <- subset(B, idents = 'B/plasma')

Idents(B) <- 'subcluster'

unique(Idents(B))

B <- subset(B, idents=c('activated_T','CD8_T','γδT','CD4+T'),invert=TRUE)

endo <- readRDS('./for_analysis/論文用再検討/修正/data_9_1_annotated_endothelial.rds')

### ref matrix, using scRNA-seq expression data (only epithelial cells and fibroblasts)

In [ ]:
ref <- c()

In [ ]:
for (i in 1:length(unique(epi@meta.data$subcluster))){
    clus <- unique(epi@meta.data$subcluster)[i]
    aa <- subset(epi, idents=clus)
    test <- as.matrix((GetAssayData(aa)))
    me <- as.matrix(apply(test,1,mean))
    colnames(me) <- clus
    ref <- cbind(ref, me)
    }

In [ ]:
for (i in 1:length(unique(Idents(fb)))){
    clus <- unique(Idents(fb))[i]
    aa <- subset(fb, idents=clus)
    test <- as.matrix((GetAssayData(aa)))
    me <- as.matrix(apply(test,1,mean))
    colnames(me) <- clus
    ref <- cbind(ref, me)
    }

for (i in 1:length(unique(Idents(B)))){
    clus <- unique(Idents(B))[i]
    aa <- subset(B, idents=clus)
    test <- as.matrix((GetAssayData(aa)))
    me <- as.matrix(apply(test,1,mean))
    colnames(me) <- clus
    ref <- cbind(ref, me)
    }

for (i in 1:length(unique(Idents(endo)))){
    clus <- unique(Idents(endo))[i]
    aa <- subset(endo, idents=clus)
    test <- as.matrix((GetAssayData(aa)))
    me <- as.matrix(apply(test,1,mean))
    colnames(me) <- clus
    ref <- cbind(ref, me)
    }

In [ ]:
head(ref)

In [ ]:
colnames(ref)

In [ ]:
1+1

In [ ]:
# A few LEFTY1+cell so exclude from reference
ref <- ref[,-10][,-12]

In [ ]:
ref <- ref[,-6]

In [ ]:
ls()

In [ ]:
colnames(ioprofiles)

In [ ]:
head(ioprofiles)

In [ ]:
# fibroblast is included in our scRNA-seq
ioprofiles <- ioprofiles[,-3]

#### 不要になったコード

raw expression of cosmx
cts <- get_expression_values(fov_join)

cts <- cts@exprMat

Mean value of negative probes in each cell
negmean <- Matrix::colMeans(cts[rownames(cts)[grepl('^Neg',rownames(cts))],])

#### 再開

In [ ]:
counts_all <- Giotto:::get_expression_values(fov_join, feat_type = 'rna')

In [ ]:
counts_all <- counts_all@exprMat

In [ ]:
# ** Note: `counts_all[]` is shorthand for `counts_all@exprMat` **

# extract the raw matrix
counts_feat <- counts_all[rownames(counts_all)[!grepl('^Neg',rownames(counts_all ))],]
counts_neg <- counts_all[rownames(counts_all)[grepl('^Neg',rownames(counts_all ))],]

In [ ]:
# calculate the mean of negative probe per cell
negmean <- Matrix::colMeans(counts_neg[])

In [ ]:
counts <- as.matrix(counts_feat)

In [ ]:
counts <- t(counts)

In [ ]:
length(negmean)

In [ ]:
length(rowSums(counts))

In [ ]:
negmean.per.totcount <- mean(negmean) / mean(rowSums(counts))

In [ ]:
per.cell.bg <- rowSums(counts) * negmean.per.totcount

In [ ]:
dim(counts_feat)

In [ ]:
head(negmean)

In [ ]:
dim(ioprofiles)

In [ ]:
# coioprofilesstatistics used to define anchor cells.
# (This step is slow for big datasets. It's recommended to run this once and save the results. 
# Then you can iteratively adjust your choices when selecting anchor cells)
astats <- get_anchor_stats(counts = counts,
                           neg = negmean,
                           profiles = ioprofiles
                          )

In [ ]:
# now choose anchors:
anchors <- choose_anchors_from_stats(counts = counts, 
                                     neg = negmean, 
                                     bg = per.cell.bg,
                                     anchorstats = astats, 
                                     # a very low value chosen for the mini
                                     # dataset. Typically hundreds of cells
                                     # would be better.
                                     n_cells = 500, 
                                     min_cosine = 0.4, 
                                     min_scaled_llr = 0.03, 
                                     insufficient_anchors_thresh = 5)

In [ ]:
updatedprofiles <- updateReferenceProfiles(reference_profiles = ioprofiles, 
                                           counts = counts, 
                                           neg = negmean, 
                                           bg = per.cell.bg,
                                           anchors = anchors) 
str(updatedprofiles)
#> List of 2
#>  $ updated_profiles: num [1:960, 1:5] 0 0 0.019 0 0 0.019 0.039 0 0.019 0.059 ...
#>   ..- attr(*, "dimnames")=List of 2
#>   .. ..$ : chr [1:960] "AATK" "ABL1" "ABL2" "ACE" ...
#>   .. ..$ : chr [1:5] "plasmablast" "macrophage" "fibroblast" "endothelial" ...
#>  $ anchors         : Named chr [1:2198] NA NA NA NA ...
#>   ..- attr(*, "names")= chr [1:2198] "c_3_18_2" "c_3_18_3" "c_3_18_4" "c_3_18_5" ...

In [ ]:
# compute the statistics used to define anchor cells.
# (This step is slow for big datasets. It's recommended to run this once and save the results. 
# Then you can iteratively adjust your choices when selecting anchor cells)
astats2 <- get_anchor_stats(counts = counts,
                           neg = negmean,
                           profiles = ref)

In [ ]:
# now choose anchors:
anchors2 <- choose_anchors_from_stats(counts = counts, 
                                     neg = negmean, 
                                     bg = per.cell.bg,
                                     anchorstats = astats2, 
                                     # a very low value chosen for the mini
                                     # dataset. Typically hundreds of cells
                                     # would be better.
                                     n_cells = 500, 
                                     min_cosine = 0.4, 
                                     min_scaled_llr = 0.03, 
                                     insufficient_anchors_thresh = 5)

In [ ]:
updatedprofiles2 <- updateReferenceProfiles(reference_profiles = ioprofiles2, 
                                           counts = counts, 
                                           neg = negmean, 
                                           bg = per.cell.bg,
                                           anchors = anchors2) 
#str(updatedprofiles)
#> List of 2
#>  $ updated_profiles: num [1:960, 1:5] 0 0 0.019 0 0 0.019 0.039 0 0.019 0.059 ...
#>   ..- attr(*, "dimnames")=List of 2
#>   .. ..$ : chr [1:960] "AATK" "ABL1" "ABL2" "ACE" ...
#>   .. ..$ : chr [1:5] "plasmablast" "macrophage" "fibroblast" "endothelial" ...
#>  $ anchors         : Named chr [1:2198] NA NA NA NA ...
#>   ..- attr(*, "names")= chr [1:2198] "c_3_18_2" "c_3_18_3" "c_3_18_4" "c_3_18_5" ...

In [ ]:
# bind the reference matrix
updatedprofiles_cbind <- cbind(updatedprofiles$updated_profiles, updatedprofiles2$updated_profiles)

### semi-supervised clustering

#### memory consuming

you can check memory use by "ps -o pid,user,%mem,command ax | sort -b -k3 -r | head -10" and "docker state"

In [ ]:
saveRDS(updatedprofiles_cbind, 'updatedprofiles.rds')

In [ ]:
updatedprofiles_cbind <- readRDS('updatedprofiles.rds')

### count=0不要ならOK

In [ ]:
meta <- pDataDT(fov_join)

In [ ]:
length(meta$cell_ID)

In [ ]:
immunofluordata <- cbind(meta$Mean.CD3, meta$Mean.CD45, meta$Mean.MembraneStain, meta$Mean.PanCK,meta$Area, meta$AspectRatio)

In [ ]:
rownames(immunofluordata) <- meta$cell_ID

In [ ]:
# perform automatic cohorting:
cohort <- fastCohorting(immunofluordata,
                        gaussian_transform = TRUE) 
# ("Gaussian_transform = TRUE" maps variables to gaussians in order to 
#  place dramatically different variables on the same scale.)
table(cohort)
#> cohort
#>    1    2    3 
#> 1236  631  331

In [ ]:
length(cohort)

In [ ]:
semisup <- insitutype(  x = counts,
  neg = negmean,
  cohort = cohort,
  # Enter your own per-cell background estimates here if you
  # have them; otherwise insitutype will use the negprobes to
  # estimate background for you.
  bg = NULL,
  # condensed to save time. n_clusts = 5:15 would be more optimal
  n_clusts = c(2, 10),
  reference_profiles = updatedprofiles_cbind,
  update_reference_profiles = FALSE,
  # choosing inadvisably low numbers to speed the vignette; using the defaults
  # in recommended.
)

In [ ]:
length(semisup)

In [ ]:
length(semisup$clust)

In [ ]:
1+1

In [ ]:
z <- Sys.time()

In [ ]:
z

### supervised clustering

In [ ]:
sup <- insitutypeML(x = counts,
  neg = negmean,
  cohort = cohort,
  # Enter your own per-cell background estimates here if you
  # have them; otherwise insitutype will use the negprobes to
  # estimate background for you.
  #bg = NULL,
  # condensed to save time. n_clusts = 5:15 would be more optimal
#  n_clusts = c(2, 10),
  reference_profiles = updatedprofiles_cbind,
  #update_reference_profiles = FALSE,
  # choosing inadvisably low numbers to speed the vignette; using the defaults
  # in recommended.
) 

In [ ]:
str(semisup)
#> List of 5
#>  $ clust                      : Named chr [1:2198] "c" "c" "c" "c" ...
#>   ..- attr(*, "names")= chr [1:2198] "c_3_18_2" "c_3_18_3" "c_3_18_4" "c_3_18_5" ...
#>  $ prob                       : Named num [1:2198] 1 1 0.95 1 1 ...
#>   ..- attr(*, "names")= chr [1:2198] "c_3_18_2" "c_3_18_3" "c_3_18_4" "c_3_18_5" ...
#>  $ profiles                   : num [1:960, 1:11] 0 0.0291 0.0382 0.0382 0.02 ...
#>   ..- attr(*, "dimnames")=List of 2
#>   .. ..$ : chr [1:960] "AATK" "ABL1" "ABL2" "ACE" ...
#>   .. ..$ : chr [1:11] "a" "b" "c" "d" ...
#>  $ logliks                    : num [1:2198, 1:11] -127 -134 -337 -141 -300 ...
#>   ..- attr(*, "dimnames")=List of 2
#>   .. ..$ : chr [1:2198] "c_3_18_2" "c_3_18_3" "c_3_18_4" "c_3_18_5" ...
#>   .. ..$ : chr [1:11] "a" "b" "c" "d" ...
#>  $ logliks_from_lost_celltypes: num[1:2198, 0 ] 
#>   ..- attr(*, "dimnames")=List of 2
#>   .. ..$ : chr [1:2198] "c_3_18_2" "c_3_18_3" "c_3_18_4" "c_3_18_5" ...
#>   .. ..$ : NULL
round(head(semisup$prob), 2)
#> c_3_18_2 c_3_18_3 c_3_18_4 c_3_18_5 c_3_18_6 c_3_18_7 
#>     1.00     1.00     0.95     1.00     1.00     1.00
heatmap(sweep(semisup$profiles, 1, pmax(apply(semisup$profiles, 1, max), .2), "/"), scale = "none",
        main = "Mean cell type expression profiles")

In [ ]:
dim(fov_join@cell_metadata$cell$rna)

In [ ]:
saveRDS(semisup, '20221205_semisup_cls_cosmx_epi_immune.rds')

#### この時点でUMAP、x,y情報なし

In [ ]:
semisup <- readRDS('20221205_semisup_cls_cosmx_epi_immune.rds')

In [ ]:
length(semisup$clust)

In [ ]:
head(semisup$clust)

In [ ]:
head(pDataDT(fov_join)$cell_ID)

In [ ]:
length(semisup$clust[pDataDT(fov_join)$cell_ID])

In [ ]:
head(semisup$logliks)

In [ ]:
cols <- readRDS('./integrated_data/論文用再検討/distinct_colors.Rds')

In [ ]:
library(InSituType)

In [ ]:
head(semisup$profiles)

In [ ]:
# make the flightpath plot
options(repr.plot.width=7, repr.plot.height=7)
fp <- flightpath_plot(flightpath_result = NULL, insitutype_result = semisup)
class(fp)

In [ ]:
fp_layout <- flightpath_layout(logliks = semisup$logliks, profiles = semisup$profiles)

In [ ]:
str(fp_layout)

In [ ]:
head(semisup$logliks)

In [ ]:
head(semisup$profiles)

In [ ]:
dim(fp_layout$clustpos)

In [ ]:
head(fp_layout$cellpos)

In [ ]:
fp

In [ ]:
saveGiotto(fov_join, foldername = 'saveGiottoDir_final_before_submission')

# 7. Calculate dimension reduction

In [ ]:
library(dplyr)

In [ ]:
# typical way of calculating HVG
options(repr.plot.width=7, repr.plot.height=7)
fov_join <- calculateHVF(gobject = fov_join,
                         HVFname = 'hvg_orig')

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
# new method based on variance of pearson residuals for each gene
fov_join <- calculateHVF(gobject = fov_join,
                         method = 'var_p_resid',
                         expression_values = 'pearson',
                         show_plot = T)

In [ ]:
gene_meta = fDataDT(fov_join)

In [ ]:
length(gene_meta[hvf == 'yes']$feat_ID)

In [ ]:
gene_meta[hvf == 'yes']$feat_ID

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
fov_join <- runPCA(gobject = fov_join, feats_to_use = gene_meta[hvf == 'yes']$feat_ID,
                   expression_values = 'pearson',
                   scale_unit = F,
                   center = F)
screePlot(fov_join, ncp = 20)

In [ ]:
plotPCA(fov_join,
        dim1_to_use = 1,
        dim2_to_use = 2)

In [ ]:
fov_join <- runUMAP(fov_join, #min_dist = 0,
                    dimensions_to_use = 1:20,
                    n_threads = 4)


In [ ]:
plotUMAP(gobject = fov_join)

In [ ]:
fov_join <- createNearestNetwork(gobject = fov_join,
                                 dimensions_to_use = 1:20,
                                 k = 10)

# 10. Cluster

### Integrate cluster with Nanostring analysis

In [ ]:
aaa <- readRDS('seurat_object.Rds')

In [ ]:
aaa

In [ ]:
library(Seurat)

In [ ]:
unique(aaa@meta.data$fov)

In [ ]:
metadata <- aaa@meta.data

In [ ]:
library(stringr)

In [ ]:
# replace the fov_join 
# take some time 
test10 <- c()
for (i in 1:dim(fov_join@cell_metadata$cell$rna )[1]){
    test <- fov_join@cell_metadata$cell$rna$cell_ID[i]
        if (str_detect(test, 'Fundic')) {
            test2 <- str_replace(test, 'Fundic-','Run5458.5006_')
            test3 <- str_split(test2, '_')[[1]]
            test4 <- paste0(test3[1], '_',test3[3],'_',test3[2])
            j <- as.integer(str_sub(test3[2],4,6))
#            print(test3[2])
 #           print(test4)
            
        if ( j < 10) {
        test4 <- str_replace(test4, paste0('fov00',j,'-cell'), as.character(j))
            } else {
            test4 <- str_replace(test4, paste0('fov0',j,'-cell'), as.character(j))
            }    
        } else {
            test2 <- str_replace(test, 'Pyloric-','Run5458.0507_')
            test3 <- str_split(test2, '_')[[1]]
            test4 <- paste0(test3[1], '_',test3[3],'_',test3[2])
            j <- as.integer(str_sub(test3[2],4,6))
#            print(test3[2])
 #           print(test4)
            
        if ( j < 10) {
        test4 <- str_replace(test4, paste0('fov00',j,'-cell'), as.character(j))
            } else {
            test4 <- str_replace(test4, paste0('fov0',j,'-cell'), as.character(j))
            }    
        }
            
            
    test10 <- c(test10,test4)
    }

In [ ]:
length(test10)

In [ ]:
head(test10)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = test10, vector_name = 'cell_ID_2')

In [ ]:
giotto_meta <- pDataDT(fov_join)

In [ ]:
colnames(giotto_meta)

In [ ]:
giotto_meta <- as.data.frame(giotto_meta)

In [ ]:
rownames(giotto_meta) = giotto_meta$cell_ID_2

In [ ]:
dim(giotto_meta)

In [ ]:
dim(metadata2)

In [ ]:
head(rownames(metadata2))

In [ ]:
inter <- intersect(rownames(metadata2), rownames(giotto_meta))

In [ ]:
length(inter)

In [ ]:
giotto_meta <- giotto_meta[inter,]

In [ ]:
subset_cell_ids = giotto_meta$cell_ID

In [ ]:
dim(pDataDT(fov_join2))

In [ ]:
fov_join <- fov_join2

In [ ]:
remove(fov_join2)

In [ ]:
min(metadata2$nCount_Nanostring)

In [ ]:
dim(metadata2)

fov_join <- addCellMetadata(fov_join, new_metadata = metadata2)

In [ ]:
colnames(pDataDT(fov_join))

In [ ]:
table(pDataDT(fov_join)$tissue)

In [ ]:
sum(pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_0507']$nCount_Nanostring)

In [ ]:
max(pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_0507']$nCount_Nanostring)

In [ ]:
max(pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_0507']$Area)*0.324

In [ ]:
a <- pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_0507']

In [ ]:
a[a$list_ID=='Fundic']

In [ ]:
mean(pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_0507']$Area)

In [ ]:
19180536/129636

In [ ]:
sum(pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_5006']$nCount_Nanostring)

In [ ]:
max(pDataDT(fov_join)[pDataDT(fov_join)$tissue=='Stomach_5006']$nCount_Nanostring)

In [ ]:
19180536/129636

In [ ]:
options(repr.plot.height=7, repr.plot.width=7)
cellmeta = pDataDT(fov_join, feat_type = 'rna')
hist(cellmeta$nr_feats, 100)

In [ ]:
cell_ids <- colnames(fov_join@expression$cell$rna$raw)

### add InSituType cluster

In [ ]:
head(semisup$clust)

In [ ]:
head(pDataDT(fov_join))

In [ ]:
head(semisup$clust)

In [ ]:
head(pDataDT(fov_join)$cell_ID)

In [ ]:
semisup$clust[pDataDT(fov_join)$cell_ID]

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = as.data.frame(semisup$clust[pDataDT(fov_join)$cell_ID]))

In [ ]:
unique(fov_join@cell_metadata$cell$rna$'semisup$clust[pDataDT(fov_join)$cell_ID]')

In [ ]:
# visualize UMAP cluster results
options(repr.plot.width=14,repr.plot.height=14)
plotUMAP(gobject = fov_join,
         cell_color = 'semisup$clust[pDataDT(fov_join)$cell_ID]',
         show_NN_network = FALSE,
         point_size = 1)

In [ ]:
cols <- unlist(cols[seq_along(unique(semisup$clust))])
names(cols) <- unique(semisup$clust)
cols[is.element(names(cols), names(iocolors))] <- iocolors[names(cols)[is.element(names(cols), names(iocolors))]]

par(mfrow = c(1, 2))
par(mar = c(0, 0, 3, 0))

plot(fov_join@spatial_locs$cell$raw$sdimx, fov_join@spatial_locs$cell$raw$sdimy, pch = 16, cex = .01, asp = 1, cex.main = 0.75,
          main = "cells in physical space",
     col = cols[semisup$clust], xlab = "", ylab = "", xaxt = "n", yaxt = "n")

plot(fov_join@dimension_reduction$cells$cell$rna$umap$umap@coordinates, pch = 16, cex = .01, asp = 1, cex.main = 0.75,
     main = "cells in UMAP space",     
     col = cols[semisup$clust], xlab = "", ylab = "", xaxt = "n", yaxt = "n")
legend("bottomleft", pch = 16, col = cols, legend = names(cols), cex = 0.7)

fov_join <- doLeidenCluster(gobject = fov_join,
                            resolution = 0.1,
                            n_iterations = 1000)

# CosMx cluster annotation

In [ ]:
library(Giotto)
library(data.table)
library(Seurat)

fov_join <- loadGiotto('./saveGiottoDir_final_before_submission/')

In [ ]:
head(pDataDT(fov_join))

In [ ]:
colnames(fov_join@cell_metadata$cell$rna@metaDT)[28] <- 'cell_types'

In [ ]:
# take few hours and use a lot of memory
markers2 = findMarkers_one_vs_all(gobject = fov_join,
                                 method = 'gini',
                                 expression_values = 'normalized',
                                 cluster_column = 'cell_types2',
                                 min_feats = 5,
                                 rank_score = 2)
# violinplot

spatDimPlot2D(gobject = fov_join,
              show_image = T,
              image_name = image_names,
              cell_color = 'cell_types2',
              spat_point_size = 0.5)

In [ ]:
write.csv(markers,'20221205_cosmx_marker_gene.csv')

In [ ]:
markers <- read.csv('20221205_cosmx_marker_gene.csv')

In [ ]:
library(dplyr)

In [ ]:
head(markers)

markers %>% group_by(cluster) %>% top_n(3, expression_gini)

In [ ]:
topgini_genes <- markers %>% group_by(cluster) %>% top_n(3, expression_gini)

In [ ]:
topgini_genes = unique(topgini_genes$feats)

In [ ]:
cluster_order = unique(fov_join@cell_metadata$cell$rna$cell_types)

In [ ]:
options(repr.plot.width=14, repr.plot.height=20)
plotMetaDataHeatmap(fov_join, 
                    expression_values = 'scaled',
                    metadata_cols = 'cell_types',
                    selected_feats = c(topgini_genes,'KRT19','MKI67','EPCAM','LEFTY1','OLFM4','APOA1','CD3D','COL1A1','BMP4'),
                    custom_cluster_order = sort(cluster_order))

In [ ]:
cluster_order = sort(cluster_order)

In [ ]:
cluster_order

In [ ]:
## add cell types ###
cluster_order = sort(cluster_order)
clusters_cell_types_lung = c('TA_Ent', 'ADH1+GKN1-F', 'pro_B-cell', 'B-cell',#a,ADH1+F,b,B-cell
                             'Chief', 'CCL11+FB','Pyloric_deep','LEFTY1_epi', #c, CCL11+FB, chief, d
                             'OLFM4+Epi','endothelial','Ent','Stromal', #e,endothelial,Ent,f
                             'FibSmo','Ent2', #'FibSmo','g'
                             'GKN+F', 'Gob', 'plasma', 'plasma', #'GKN+F', 'Gob', 'h', 'i'
                             'Foveolar','KLF+FB',#j, KLF+FB
                             "macrophage","mast","mDC","pericyte",#"macrophage","mast","mDC","MFB"
                             'monocyte','NE1','NE2','neutrophil',
                             'NK','PC','pDC','PDGFR+FB','PG/Neck1', 'PG/Neck2', 'plasma','pericyte_SM','T CD4 memory', 'T CD4 naive', 'T CD8 memory','T CD8 naive','Treg')

names(clusters_cell_types_lung) = as.character(sort(cluster_order))

In [ ]:
clusters_cell_types_lung

41 cluster -> 39 cluster

In [ ]:
head(pDataDT(fov_join))

In [ ]:
head(pDataDT(fov_join))

In [ ]:
trans <- data.frame(
    old = names(clusters_cell_types_lung),
    new = clusters_cell_types_lung
    )

In [ ]:
new_clus <- lapply(pDataDT(fov_join)$cell_types, function(x) {
    as.vector(trans$new[match(x, trans$old)])
    }
       )

In [ ]:
new_clus <- unlist(new_clus)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = new_clus)

In [ ]:
options(repr.plot.width=14, repr.plot.height=20)
p1 <- plotMetaDataHeatmap(fov_join, 
                    expression_values = 'scaled',
                    metadata_cols = 'cell_types',
                    selected_feats = c(topgini_genes,'KRT19','MKI67','EPCAM','LEFTY1','OLFM4','APOA1','CD3D','COL1A1','BMP4'),
                    custom_cluster_order = sort(cluster_order))

In [ ]:
length(unique(pDataDT(fov_join)$new_clus))

In [ ]:
cluster_order2 <- c('Foveolar',
                                           'GKN+F',
                                           'ADH1+GKN1-F',
                    'Ent',
                                           'TA_Ent',
                                           'Ent2',
                                           'OLFM4+Epi',
                                           'Gob',
                                           'NE1',
                                           'NE2',
                                           'LEFTY1_epi',
                                           'PG/Neck1',
                                           'Chief',
                                           'PC',
                                           'PG/Neck2',
                                           'Pyloric_deep',
                                           
                                           'B-cell',
                                           'pro_B-cell',
                                           'plasma',
                                           'T CD4 memory',
                                           'T CD8 memory',
                                           'T CD4 naive',
                                           'T CD8 naive',
                                           'Treg',
                                           'NK',
                                           'macrophage',
                                           'mast',
                                           'mDC',
                                           'monocyte',
                                           'pDC',
                                           'neutrophil',
                    'endothelial',
                                           'CCL11+FB',
                                           'KLF+FB',
                                           'PDGFR+FB',
                                           'pericyte',
                                           'FibSmo',
                                           'Stromal',
                                           'pericyte_SM'
                                           )

In [ ]:
options(repr.plot.width=14, repr.plot.height=20)
p2 <- plotMetaDataHeatmap(fov_join, show_plot = FALSE,return_plot = TRUE,
                    expression_values = 'scaled',
                    metadata_cols = 'new_clus',
                    selected_feats = c(topgini_genes,'KRT19','MKI67','EPCAM','LEFTY1','OLFM4','APOA1','CD3D','COL1A1','BMP4'),
                    custom_cluster_order = rev(cluster_order2))

In [ ]:
class(p2)

In [ ]:
library(ggplot2)

In [ ]:
options(repr.plot.width=16, repr.plot.height=10)
pdf('Fig6C.pdf',width = 16, height=10)
p2 + coord_flip() + theme(axis.text.x = element_text(angle = 90))
dev.off()

In [ ]:
head(pDataDT(fov_join))

In [ ]:
table(pDataDT(fov_join)$new_clus)

In [ ]:
## add cell types ###
cluster_order = sort(cluster_order)
clusters_cell_types_lung = c('Epi', 'Epi', 'Stromal', 'Stromal',#a,ADH1+F,b,B-cell
                             'Epi', 'Stromal','Epi','Epi', #c, CCL11+FB, chief, d
                             'Epi','Stromal','Epi','Stromal', #e,endothelial,Ent,f
                             'Stromal','Epi', #'FibSmo','g'
                             'Epi', 'Epi', 'Stromal', 'Stromal', #'GKN+F', 'Gob', 'h', 'i'
                             'Epi','Stromal',#j, KLF+FB
                             "Stromal","Stromal","Stromal","Stromal",#"macrophage","mast","mDC","MFB"
                             'Stromal','Epi','Epi','Stromal',
                             'Stromal','Epi','Stromal','Stromal','Epi', 'Epi', 'Stromal','Stromal','Stromal', 'Stromal', 'Stromal','Stromal','Stromal')

names(clusters_cell_types_lung) = as.character(sort(cluster_order))

In [ ]:
clusters_cell_types_lung

In [ ]:
head(pDataDT(fov_join))

In [ ]:
trans <- data.frame(
    old = names(clusters_cell_types_lung),
    new = clusters_cell_types_lung
    )

In [ ]:
major_clus <- lapply(pDataDT(fov_join)$cell_types, function(x) {
    as.vector(trans$new[match(x, trans$old)])
    }
       )

In [ ]:
major_clus <- unlist(major_clus)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = major_clus)

In [ ]:
## add cell types ###
cluster_order = sort(cluster_order)
clusters_cell_types_lung = c('Epi', 'Epi', 'B/plasma', 'B/plasma',#a,ADH1+F,b,B-cell
                             'Epi', 'Fibro','Epi','Epi', #c, CCL11+FB, chief, d
                             'Epi','endothelial','Epi','Stromal', #e,endothelial,Ent,f
                             'Fibro','Epi', #'FibSmo','g'
                             'Epi', 'Epi', 'B/plasma', 'B/plasma', #'GKN+F', 'Gob', 'h', 'i'
                             'Epi','Fibro',#j, KLF+FB
                             "MC","MC","MC","Fibro",#"macrophage","mast","mDC","MFB"
                             'MC','Epi','Epi','MC',
                             'NK','Epi','MC','Fibro','Epi', 'Epi', 'B/plasma','Fibro','T', 'T', 'T','T','T')

names(clusters_cell_types_lung) = as.character(sort(cluster_order))

In [ ]:
clusters_cell_types_lung

In [ ]:
head(pDataDT(fov_join))

In [ ]:
trans <- data.frame(
    old = names(clusters_cell_types_lung),
    new = clusters_cell_types_lung
    )

In [ ]:
major_clus2 <- lapply(pDataDT(fov_join)$cell_types, function(x) {
    as.vector(trans$new[match(x, trans$old)])
    }
       )

In [ ]:
major_clus2 <- unlist(major_clus2)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = major_clus2)

# For Seurat Object

In [ ]:
library(Seurat)
library(dplyr)

In [ ]:
ls()

In [ ]:
counts <- get_expression_values(fov_join)

In [ ]:
#aaa <- readRDS('seurat_object.Rds')

cosmx <- CreateSeuratObject(counts = counts@exprMat, project='cosmx')

In [ ]:
giotto_meta <- as.data.frame(pDataDT(fov_join))

In [ ]:
head(pDataDT(fov_join))

In [ ]:
rownames(giotto_meta) <- giotto_meta$cell_ID

In [ ]:
for (i in colnames(pDataDT(fov_join))){
    cosmx <- AddMetaData(cosmx, metadata = giotto_meta[i], col.name=i)
    }

head(aaa[[]])

In [ ]:
head(cosmx[[]])

In [ ]:
metadata2 <- cosmx@meta.data

In [ ]:
rownames(metadata2) <- cosmx@meta.data$id

In [ ]:
umaps <- fov_join@dimension_reduction$cells$cell$rna$umap$umap

In [ ]:
umaps <- umaps@coordinates

In [ ]:
umaps <- as.data.frame(umaps)

table(aaa@meta.data['tissue'])

umap_a <- umaps[1:129941,]

umap_b <- umaps[129942:244750,]

umaps <- rbind(umap_b, umap_a)

head(umaps)

cosmx <- AddMetaData(cosmx, metadata = umaps$umap_1, col.name = 'umap_1')

cosmx <- AddMetaData(cosmx, metadata = umaps$umap_2, col.name = 'umap_2')

head(cosmx[[]])

umaps <- cbind(cosmx[[]]$umap_1,cosmx[[]]$umap_2)

In [ ]:
colnames(umaps) <- c('umap_1', 'umap_2')
rownames(umaps) <- rownames(cosmx[[]])

In [ ]:
options( repr.plot.width=7, repr.plot.height=7)
cosmx[["umap"]] <- CreateDimReducObject(embeddings = as.matrix(umaps))

In [ ]:
dim(cosmx@reductions$umap@cell.embeddings)

In [ ]:
head(fov_join@cell_metadata$cell$rna$cell_ID)

In [ ]:
head(colnames(fov_join@cell_metadata$cell$rna))

cosmx <- FindVariableFeatures(cosmx, selection.method = "vst")

cosmx <- ScaleData(cosmx)

cosmx <- RunPCA(cosmx)

DimPlot(cosmx, reduction = 'pca', group.by='cluster')

In [ ]:
spatial <- fov_join@spatial_locs$cell$raw

spatial <- spatial@coordinates

head(spatial)

rownames(spatial) <- spatial$cell_ID

spatial <- spatial[,1:2]

In [ ]:
# visualize UMAP cluster results
options(repr.plot.width=7,repr.plot.height=7)
plotUMAP(gobject = fov_join,
         cell_color = 'new_clus',
                                 cell_color_code = color_set,legend_symbol_size=8,
         show_NN_network = FALSE,
         point_size = 1)

In [ ]:
plotUMAP(fov_join)

umap <- fov_join@spatial_locs$cell$raw

spatial <- spatial@coordinates

head(spatial)

rownames(spatial) <- spatial$cell_ID

spatial <- spatial[,1:2]

In [ ]:
head(cosmx@meta.data)

In [ ]:
colnames(spatial) <- c('tsne_1', 'tsne_2')

spatial <- as.matrix(spatial)

spatial <- spatial/10000

rownames(spatial) <- rownames(cosmx[[]])

cosmx[["tsne"]] <- CreateDimReducObject(embeddings = spatial)

In [ ]:
options( repr.plot.width=14, repr.plot.height=7)
DimPlot(cosmx, reduction = 'tsne', pt.size=0.01, group.by = 'new_clus')

In [ ]:
options( repr.plot.width=14, repr.plot.height=7)
DimPlot(cosmx, reduction = 'umap', pt.size=0.01, group.by = 'new_clus')

In [ ]:
head(cosmx@meta.data)

In [ ]:
saveRDS(cosmx, './cosmx_seurat_object/cosmx_all_seurat_final_before_submission.rds')

# Sandbox

In [ ]:
table(pDataDT(fov_join)$new_clus)

### After load the save Giotto file
#### 保存したGiotto object使っていなければスキップ
#### （うまくいったが、show_image=TRUEにするとエラー(raw imageはとれていない？))

In [ ]:
cols <- readRDS('integrated_data/論文用再検討/distinct_colors.Rds')

In [ ]:
library(Giotto)

In [ ]:
packageVersion(Giotto)

In [ ]:
fov_join <- loadGiotto('./saveGiottoDir_final_before_submission//')

# subset Giotto to see the specific region

In [ ]:
color_set <- c('Foveolar'='#FFFFCC',
                                           'GKN+F'='#C7E9B4',
                                           'ADH1+GKN1-F'='#7FCDBB',
                                           'PG/Neck1'='#41B6C4',
                                           'Chief'='#2C7FB8',
                                           'PC'='#253494',
                                           'PG/Neck2'='#31A354',
                                           'Pyloric_deep'='#006837',
                                           'Ent'='#FEE5D9',
                                           'TA_Ent'='#FB6A4A',
                                           'Ent2'='#FCAE91',
                                           'OLFM4+Epi'='#CB181D',
                                           'Gob'='blue',
                                           'NE1'='Salmon',
                                           'NE2'='yellow',
                                           'LEFTY1_epi'='red',
                                           'B-cell'='#87CEEB',
                                           'pro_B-cell'='#90EE90',
                                           'plasma'='#DDA0DD',
                                           'T CD4 memory'='#696969',
                                           'T CD8 memory'='#556B2F',
                                           'T CD4 naive'='#696969',
                                           'T CD8 naive'='#556B2F',
                                           'Treg'='#800000',
                                           'NK'='black',
                                           'macrophage'='#F0E68C',
                                           'mast'='#F0E68C',
                                           'mDC'='#F0E68C',
                                           'monocyte'='#F0E68C',
                                           'pDC'='#F0E68C',
                                           'neutrophil'='#EE82EE',
                                           'CCL11+FB'='#4682B4',
                                           'KLF+FB'='Cyan',
                                           'PDGFR+FB'='Gray',
                                           'pericyte'='pink',
                                           'FibSmo'='#FFD700',
                                           'Stromal'='#ADFF2F',
                                           'pericyte_SM'='#FAFAD2',
                                           'endothelial'='#B22222')

# very small region

In [ ]:
fov_join@images$`Fundic-fov011-image`

In [ ]:
#subset a Giotto object based on spatial locations
smallfov_sub <- subsetGiottoLocs(fov_join,
                             x_max = 19500,
                             x_min = 18500,
                             y_max = 22000,
                             y_min = 21000)


In [ ]:
#extract all genes observed in new object
smallfeats <- fDataDT(fov_join)[, feat_ID]

In [ ]:
# plot only the polygon outlines
spatInSituPlotPoints(smallfov_sub,image_name = 'Fundic-fov011-image',
                     polygon_line_size = 0.1,
                     polygon_alpha = 0,
                     polygon_color = 'white',
                     show_polygon = FALSE,
                     show_image = TRUE,
                     show_legend = FALSE
                     )

In [ ]:
# plot only the polygon outlines
spatInSituPlotPoints(smallfov_sub,image_name = 'Fundic-fov011-image',
                     polygon_line_size = 1,
                     polygon_alpha = 0,
                     polygon_color = 'white',
                     show_polygon = TRUE,
                     show_image = TRUE,
                     show_legend = FALSE
                     )

In [ ]:
#plot all genes
spatInSituPlotPoints(smallfov_sub,image_name = 'Fundic-fov011-image',
                     feats = list(smallfeats),
                     point_size = 0.5,
                     polygon_line_size = 1,
                     show_polygon = TRUE,
                     polygon_color = 'white',
                     show_image = TRUE,
                     show_legend = FALSE)

In [ ]:
#plot all genes
options(repr.plot.width=14, repr.plot.height=14)
spatInSituPlotPoints(smallfov_sub,image_name = 'Fundic-fov011-image',
                     feats = list('EPCAM','COL3A1','GZMA','JCHAIN','CD3D'),
                     feats_color_code = c('green','skyblue','white','red','purple'),
                     point_size = 3,
                     polygon_line_size = 1,
                     show_polygon = TRUE,
                     polygon_color = 'white',
                     show_image = TRUE,
                     show_legend = TRUE)

In [ ]:
#plot all genes
options(repr.plot.width=14, repr.plot.height=14)
spatInSituPlotPoints(smallfov_sub,image_name = 'Fundic-fov011-image',
                     feats = list('EPCAM','KRT18','KRT19','COL1A1','COL3A1','GZMA','GZMB','GNLY','JCHAIN','CD3D','CD3E','CD3G'),
                     feats_color_code = c('green','green','green','skyblue','skyblue','white','red','red','red','purple','white','white'),
                     point_size = 3,
                     polygon_line_size = 1,
                     show_polygon = TRUE,
                     polygon_color = 'white',
                     show_image = TRUE,
                     show_legend = TRUE)

In [ ]:
# plot polygons colorlabeled with leiden clusters
spatInSituPlotPoints(smallfov_sub,image_name = 'Fundic-fov011-image',polygon_alpha = 1,
                     polygon_line_size = 0.1,
                     show_polygon = TRUE,
                     polygon_fill = 'major_clus2',
                     polygon_fill_as_factor = TRUE,
                     show_image = TRUE,
                     show_legend = FALSE,
                     )

In [ ]:
1+1

In [ ]:
cols <- readRDS('./integrated_data/論文用再検討/distinct_colors.Rds')

#### see whole FOV

In [ ]:
smallfov <- subsetGiotto(fov_join, 
                         cell_ids = pDataDT(fov_join)$cell_ID[grepl('Fundic-fov019',pDataDT(fov_join)$cell_ID)]
                        )    

In [ ]:
length(sort(unique(pDataDT(fov_join)$new_clus)))

In [ ]:
library(RColorBrewer)

In [ ]:
a <- pDataDT(fov_join)$Tissue_spatial_region

In [ ]:
head(pDataDT(fov_join))

In [ ]:
names(a) <- pDataDT(fov_join)$cell_ID

In [ ]:
head(a)

In [ ]:
write.csv(a, 'Tissue_spatial_region.csv')

In [ ]:
options(repr.plot.width=14,repr.plot.height=14)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('NRG1','ERBB2','ERBB3'),
                     feats_color_code = c('blue','green','red'),
                     point_size = 1.5,
                    feat_shape_code = list('rna' = 17),
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'major_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = c('grey','lightskyblue'),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
spatInSituPlotPoints(smallfov2, show_image = TRUE, polygon_alpha = 1,legend_text = 20,image_name = 'Fundic-fov013-image',
                   #  feats = list('CHGA','APOA1','LEFTY1'),
                    # feats_color_code = c('green','yellow','black'),
                     point_size = 1,
                     show_polygon = FALSE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('CHGA','APOA1','LEFTY1'),
                    # feats_color_code = c('green','yellow','black'),
                     point_size = 1,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
color_set <- c('Foveolar'='#FFFFCC',
                                           'GKN+F'='#C7E9B4',
                                           'ADH1+GKN1-F'='#7FCDBB',
                                           'PG/Neck1'='#41B6C4',
                                           'Chief'='#2C7FB8',
                                           'PC'='#253494',
                                           'PG/Neck2'='#31A354',
                                           'Pyloric_deep'='#006837',
                                           'Ent'='#FEE5D9',
                                           'TA_Ent'='#FB6A4A',
                                           'Ent2'='#FCAE91',
                                           'OLFM4+Epi'='#CB181D',
                                           'Gob'='blue',
                                           'NE1'='Salmon',
                                           'NE2'='yellow',
                                           'LEFTY1_epi'='red',
                                           'B-cell'='#87CEEB',
                                           'pro_B-cell'='#90EE90',
                                           'plasma'='#DDA0DD',
                                           'T CD4 memory'='#696969',
                                           'T CD8 memory'='#556B2F',
                                           'T CD4 naive'='#696969',
                                           'T CD8 naive'='#556B2F',
                                           'Treg'='#800000',
                                           'NK'='black',
                                           'macrophage'='#F0E68C',
                                           'mast'='#F0E68C',
                                           'mDC'='#F0E68C',
                                           'monocyte'='#F0E68C',
                                           'pDC'='#F0E68C',
                                           'neutrophil'='#EE82EE',
                                           'CCL11+FB'='#4682B4',
                                           'KLF+FB'='Cyan',
                                           'PDGFR+FB'='Gray',
                                           'pericyte'='pink',
                                           'FibSmo'='#FFD700',
                                           'Stromal'='#ADFF2F',
                                           'pericyte_SM'='#FAFAD2',
                                           'endothelial'='#B22222')

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('EPHB2','EFNB2'),
                     feats_color_code = c('blue','orange'),
                     point_size = 6,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
smallfov2 <- subsetGiotto(fov_join, 
                         cell_ids = fov_join@cell_metadata$cell$rna$cell_ID[grepl('Fundic-fov013',fov_join@cell_metadata$cell$rna$cell_ID)])    

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov2, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('EPHB2','EFNB2'),
                     feats_color_code = c('blue','red'),
                     point_size = 6,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'black',
                     polygon_line_size = 0.1,
                     polygon_fill = 'major_clus',
                     polygon_fill_as_factor = TRUE,
                     #polygon_fill_code = rep('white',38),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
color_set

In [ ]:
col_fig <- c(color_set[1:16],'black')

In [ ]:
names(col_fig)[17] <- 'PDGFR+FB'

In [ ]:
pDataDT(fov_join)$cell_types

In [ ]:
color_set2 <- c(color_set, 'h'='#DDA0DD','i'='#DDA0DD','plasmablast'='#DDA0DD')

In [ ]:
names(color_set2) <- c('j',
                      'GKN+F',
                      'ADH1+GKN1-F',
                      'PG/Neck1',
                      'c',
                      'PC',
                      'PG/Neck2',
                      'Chief',
                      'Ent',
                      'a',
                      'g',
                      'e',
                      'Gob',
                      'NE1',
                      'NE2',
                      'd',
                      'B-cell',
                      'pro_B-cell',
                      'plasma',
                      'T CD4 memory',
                      'T CD8 memory',
                      'T CD4 naive',
                      'T CD8 naive',
                      'Treg',
                      'NK',
                      'macrophage',
                      'mast',
                      'mDC',
                      'monocyte',
                      'pDC',
                      'neutrophil',
                      'CCL+FB',
                      'KLF+FB',
                      'PDGFR+FB',
                      'pericyte',
                      'FibSmo',
                      'f',
                      'SM',
                      'endothelial')

In [ ]:
color_set

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov2, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('LEFTY1'),
                    # feats_color_code = c('black'),
                     point_size = 3,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'cell_types',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = c(color_set2),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotDensity(smallfov2, 
                     feats = list('WNT5A','BMP4','BMP2','WNT7A'),
                      cow_n_col = 2
                      )

In [ ]:
fov_join2@feat_metadata <- fov_join@feat_metadata

In [ ]:
smallfov2 <- subsetGiotto(fov_join2, 
                         cell_ids = fov_join2@cell_metadata$cell$rna$cell_ID[grepl('Fundic-fov013',fov_join2@cell_metadata$cell$rna$cell_ID)],
                         )    

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov2, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('KRT19','EPCAM','PSCA'),
                    # feats_color_code = c('pink','light green','black'),
                     point_size = 1,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'cell_types',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = unlist(cols[length(unique(smallfov2@cell_metadata$cell$rna$cell_types))]),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

saveGiotto(fov_join, foldername = 'saveGiottoDir_spatialnetwork')

# add the spatial niche information from manually annotation object

In [ ]:
aaa <- readRDS('cosmx_seurat_spatial_annotation.rds')

In [ ]:
meta <- aaa[[]]

In [ ]:
meta[meta$Tissue_spatial_region == 'Fundic?_deep',]$Tissue_spatial_region = 'Fundic_deep'

In [ ]:
meta[meta$Tissue_spatial_region == 'Submucosa2',]$Tissue_spatial_region = 'Submucosa_deep'

In [ ]:
meta[meta$Tissue_spatial_region == 'Submucosa',]$Tissue_spatial_region = 'Submucosa_superficial'

In [ ]:
aaa@meta.data['Tissue_spatial_region'] = meta$Tissue_spatial_region

In [ ]:
table(aaa@meta.data['Tissue_spatial_region'])

In [ ]:
inter <- intersect(colnames(aaa), pDataDT(fov_join)$cell_ID)

In [ ]:
head(inter)

In [ ]:
length(inter)

In [ ]:
ls()

In [ ]:
fov_join <- subsetGiotto(fov_join, cell_ids = inter)

In [ ]:
head(pDataDT(fov_join2))

In [ ]:
head(pDataDT(fov_join))

In [ ]:
aaa <- subset(aaa, cells = inter)

In [ ]:
aaa

In [ ]:
meta <- aaa[[]]

In [ ]:
head(meta)

In [ ]:
Tissue_spatial_region <- c(meta[meta$Run_Tissue_name=='Run5458_0507',]$Tissue_spatial_region,meta[meta$Run_Tissue_name=='Run5458_5006',]$Tissue_spatial_region)

In [ ]:
new_clus_ <- c(pDataDT(fov_join)[pDataDT(fov_join2)$site=='Fundic',]$new_clus,
              pDataDT(fov_join)[pDataDT(fov_join2)$site=='Pyloric',]$new_clus)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = Tissue_spatial_region)

In [ ]:
aaa@meta.data['new_clus'] <- new_clus_

saveRDS(aaa, 'cosmx_seurat_annotation_semisup_epi_fb_immune.rds')

# Network analysis

In [ ]:
## delaunay network: stats + creation
options(repr.plot.width=14,repr.plot.height=14)
plotStatDelaunayNetwork(gobject = fov_join, maximum_distance = 400, save_plot = F)
fov_join = createSpatialNetwork(gobject = fov_join, minimum_k = 2, maximum_distance_delaunay = 400)
## create spatial networks based on k and/or distance from centroid
#fov_join <- createSpatialNetwork(gobject = fov_join, method = 'kNN', k = 5, name = 'spatial_network')
fov_join <- createSpatialNetwork(gobject = fov_join, method = 'kNN', k = 10, name = 'large_network')
#fov_join <- createSpatialNetwork(gobject = fov_join, method = 'kNN', k = 100,
 #                                  maximum_distance_knn = 200, minimum_k = 2, name = 'distance_network')

In [ ]:
library(Giotto)
library(dplyr)
library(Seurat)

fov_join <- loadGiotto('./saveGiottoDir_spatialnetwork/')

In [ ]:
pDataDT(fov_join)[grepl('Fundic-fov013',pDataDT(fov_join)$cell_ID)]$cell_ID

In [ ]:
#field1_ids = cell_metadata[FOV == 0]$cell_ID
subfov_join = subsetGiotto(fov_join, cell_ids = pDataDT(fov_join)[grepl('Fundic-fov013',pDataDT(fov_join)$cell_ID)]$cell_ID)

In [ ]:
cols <- readRDS('integrated_data/論文用再検討/distinct_colors.Rds')

In [ ]:
head(pDataDT(subfov_join))

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
spatPlot(gobject = subfov_join, show_network = T, cell_color_code = color_set,
         network_color = 'blue', spatial_network_name = 'Delaunay_network',
         point_size = 2.5, cell_color = 'new_clus')

In [ ]:
spatPlot(gobject = subfov_join, show_network = T,cell_color_code = color_set,
         network_color = 'blue', spatial_network_name = 'large_network',
         point_size = 2.55, cell_color = 'new_clus')

spatGenePlot(fov_join, expression_values = 'scaled', genes = km_spatialfeats[1:4]$feats,
             point_shape = 'border', point_border_stroke = 0.1,
             show_network = F, network_color = 'lightgrey', point_size = 2.5,
             cow_n_col = 2)

heatmSpatialCorFeats(fov_join, spatCorObject = spat_cor_netw_DT, use_clus_name = 'spat_netw_clus',
                     heatmap_legend_param = list(title = NULL))

In [ ]:
ls()

In [ ]:
remove(cosmx,cell_proximities,smallfov2,smallfov_sub,p1,p2,iocolors,count_replace, counts_all, counts_feat, counts_neg, aaa)

In [ ]:
set.seed(seed =2841)
cell_proximities = cellProximityEnrichment(fov_join,
                                           cluster_column = 'new_clus',
                                           spatial_network_name = 'large_network',
                                           adjust_method= 'fdr',
                                           number_of_simulations = 1000
                                           )
                                           

In [ ]:
saveRDS(cell_proximities, file = 'cell_proximities.rds')

In [ ]:
#Barplot
cellProximityBarplot(gobject = fov_join,
         CPscore = cell_proximities,
         min_orig_ints = 100, min_sim_ints = 100)

In [ ]:
# Heatmap
cellProximityHeatmap(gobject = fov_join,
         CPscore = cell_proximities,
         color_breaks = c(-1.5, 0, 1.5),
         color_names = c('blue', 'white', 'red'))

In [ ]:
# Network
cellProximityNetwork(gobject = fov_join,
         CPscore = cell_proximities,
         remove_self_edges = T, only_show_enrichment_edges = T)

In [ ]:
# Network with self-edges
cellProximityNetwork(gobject = fov_join,
         CPscore = cell_proximities,
         remove_self_edges = T, self_loop_strength = 0.3,
         only_show_enrichment_edges = F,
         rescale_edge_weights = T,
         node_size = 8,
         edge_weight_range_depletion = c(1, 2),
         edge_weight_range_enrichment = c(2,5))

In [ ]:
head(cell_proximities)

# Network analysis without lymph etc

In [ ]:
cell_ID_mucosa <- pDataDT(fov_join)[(pDataDT(fov_join)$Tissue_spatial_region == 'Metaplasia_deep') |
                      (pDataDT(fov_join)$Tissue_spatial_region == 'Metaplasia_superficial') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Pyloric_deep') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Pyloric_ithmus') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Pyloric_to_IM_deep') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Pyloric_to_IM_ithmus') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Pyloric_to_IM_superficial') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Fundic_deep') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Fundic_ithmus') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Submucosa_superficial') |
(pDataDT(fov_join)$Tissue_spatial_region == 'Fundic_superficial')
                  ]$cell_ID

In [ ]:
fov_mucosa <- subsetGiotto(fov_join, cell_ids = cell_ID_mucosa)

In [ ]:
## delaunay network: stats + creation
options(repr.plot.width=14,repr.plot.height=14)
plotStatDelaunayNetwork(gobject = fov_mucosa, maximum_distance = 400, save_plot = F)
fov_mucosa = createSpatialNetwork(gobject = fov_mucosa, minimum_k = 2, maximum_distance_delaunay = 400)
## create spatial networks based on k and/or distance from centroid
#fov_join <- createSpatialNetwork(gobject = fov_join, method = 'kNN', k = 5, name = 'spatial_network')
fov_mucosa <- createSpatialNetwork(gobject = fov_mucosa, method = 'kNN', k = 10, name = 'large_network')
#fov_join <- createSpatialNetwork(gobject = fov_join, method = 'kNN', k = 100,
 #                                  maximum_distance_knn = 200, minimum_k = 2, name = 'distance_network')

In [ ]:
ls()

In [ ]:
packageVersion('InSituType')

In [ ]:
sessionInfo()

In [ ]:
remove(aaa,fov_join2,count_replace,counts_all,counts_feat,counts_neg)

In [ ]:
set.seed(seed =2841)
cell_proximities_mucosa = cellProximityEnrichment(fov_mucosa,
                                           cluster_column = 'new_clus',
                                           spatial_network_name = 'large_network',
                                           adjust_method= 'fdr',
                                           number_of_simulations = 1000
                                           )
                                           

In [ ]:
saveRDS(cell_proximities_mucosa, file = 'cell_proximities_mucosa.rds')

In [ ]:
cell_proximities_mucosa <- readRDS('cell_proximities_mucosa.rds')

In [ ]:
#Barplot
cellProximityBarplot(gobject = fov_mucosa,
         CPscore = cell_proximities_mucosa,
         min_orig_ints = 100, min_sim_ints = 100)

In [ ]:
#Barplot
cellProximityBarplot(gobject = fov_mucosa,
         CPscore = cell_proximities_mucosa,
         min_orig_ints = 100, min_sim_ints = 100)

In [ ]:
head(cluster_order2)

In [ ]:
# Heatmap
## visualize all genes
options(repr.plot.width=10, repr.plot.height=10)
#pdf('Figure6G.pdf',width = 7, height = 7)
p1 <- cellProximityHeatmap(gobject = fov_mucosa,return_plot = TRUE,
         CPscore = cell_proximities_mucosa,
         color_breaks = c(-1.5, 0, 1.5),
         color_names = c('blue', 'white', 'red'))
#dev.off()

In [ ]:
pdf('Figure6G.pdf',width = 10, height = 10)
p1
dev.off()

In [ ]:
# Heatmap
cellProximityHeatmap(gobject = fov_mucosa,
         CPscore = cell_proximities_mucosa,
         color_breaks = c(-1.5, 0, 1.5),
         color_names = c('blue', 'white', 'red'))

In [ ]:
# Network
cellProximityNetwork(gobject = fov_mucosa,
         CPscore = cell_proximities_mucosa,
         remove_self_edges = T, only_show_enrichment_edges = T)

In [ ]:
head(cell_proximities_mucosa)

# see the specific FOV

fov_mucosa <- createSpatialGrid(gobject = fov_mucosa,
                                sdimx_stepsize = 500,
                                sdimy_stepsize = 500,
                                minimum_padding = 50)

pDataDT(fov_mucosa)[grepl('Fundic-fov020',pDataDT(fov_mucosa)$cell_ID)]$cell_ID

In [ ]:
library(Giotto)

In [ ]:
1+1

In [ ]:
subfov_join3 = subsetGiotto(fov_join, cell_ids = pDataDT(fov_join)[grepl('Fundic-fov013',pDataDT(fov_join)$cell_ID)]$cell_ID)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
p1 <- spatInSituPlotPoints(subfov_join3, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('WNT5A','FZD5'),return_plot =TRUE,
                     feats_color_code = c('red','blue'),
                    feat_shape_code = list('rna' = 17),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'major_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = c('grey','lightblue'),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
p1 <- spatInSituPlotPoints(subfov_join3, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('IGF1','IGF1R'),return_plot =TRUE,
                     feats_color_code = c('red','blue'),
                    feat_shape_code = list('rna' = 17),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'major_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = c('grey','lightblue'),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
subfov_join = subsetGiotto(fov_join, cell_ids = pDataDT(fov_join)[grepl('Fundic-fov002',pDataDT(fov_join)$cell_ID)]$cell_ID)

In [ ]:
library(ggplot2)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
p1 <- spatInSituPlotPoints(subfov_join, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('MKI67','LEFTY1'),return_plot =TRUE,
                     feats_color_code = c('red','purple4'),
                    feat_shape_code = list('rna' = 17),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'major_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = c('grey', "grey"),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
p2 <- spatInSituPlotPoints(subfov_join, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
           #          feats = list('MKI67','LEFTY1'),return_plot =TRUE,
            #         feats_color_code = c('gold3','purple4'),
                    feat_shape_code = list('rna' = 17),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
subfov_join2 = subsetGiotto(fov_join, cell_ids = pDataDT(fov_join)[grepl('Pyloric-fov020',pDataDT(fov_join)$cell_ID)]$cell_ID)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
p2 <- spatInSituPlotPoints(subfov_join2, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                    # feats = list('MKI67','LEFTY1'),return_plot =TRUE,
                     #feats_color_code = c('red','purple4'),
                    feat_shape_code = list('rna' = 17),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
p2 <- spatInSituPlotPoints(subfov_join2, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('WNT5A','FZD5'),return_plot =TRUE,
                     feats_color_code = c('red','purple4'),
                    feat_shape_code = list('rna' = 17),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'major_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = c('grey', "lightblue"),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
p2 <- spatInSituPlotPoints(subfov_join2, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('MKI67','LEFTY1'),return_plot =TRUE,
                     feats_color_code = c('red','purple4'),
                    feat_shape_code = list('rna' = 17),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'major_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = c('grey', "grey"),
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
spec_interaction = "LEFTY1_epi--PDGFR+FB"

In [ ]:
options(repr.plot.width=14, repr.plot.height=21)
cellProximitySpatPlot(subfov_join,
                      interaction_name = spec_interaction,
                      show_network = T,
                      cluster_column = 'new_clus',
                      cell_color = 'new_clus',
                      cell_color_code = c('black','red'),  point_size_select = 3, point_size_other = 1.5)

In [ ]:
spec_interaction = "Enterocyte--NK"

In [ ]:
options(repr.plot.width=14, repr.plot.height=21)
cellProximitySpatPlot(subfov_join,
                      interaction_name = spec_interaction,
                      show_network = T,
                      cluster_column = 'new_clus',
                      cell_color = 'new_clus',
                      cell_color_code = c('black','red'),  point_size_select = 3, point_size_other = 1.5)

In [ ]:
spec_interaction = "OLFM4+Epi--NK"

In [ ]:
options(repr.plot.width=14, repr.plot.height=21)
cellProximitySpatPlot(subfov_join,
                      interaction_name = spec_interaction,
                      show_network = T,
                      cluster_column = 'new_clus',
                      cell_color = 'new_clus',
                      cell_color_code = c('black','red'),  point_size_select = 3, point_size_other = 1.5)

In [ ]:
ls()

In [ ]:
remove(subfov_join, fov_mucosa,epi)

# Individual spatial genes

In [ ]:
fov_join = createSpatialNetwork(gobject = fov_join, minimum_k = 2, maximum_distance_delaunay = 400)

In [ ]:
## 3 new methods to identify spatial genes
km_spatialfeats = binSpect(fov_join,cores = 2)

In [ ]:
ext_spatial_genes = km_spatialfeats[1:200]$feats

In [ ]:
fov_join <- createSpatialNetwork(gobject = fov_join, method = 'kNN', k = 10, name = 'large_network')

In [ ]:
## 1. calculate gene spatial correlation and single-cell correlation
## create spatial correlation object
spat_cor_netw_DT = detectSpatialCorFeats(fov_join,
                                         method = 'network',
                                         spatial_network_name = 'large_network',
                                        subset_feats = ext_spatial_genes)

In [ ]:
## 2. cluster correlated genes & visualize
spat_cor_netw_DT = clusterSpatialCorFeats(spat_cor_netw_DT,
                                          name = 'spat_netw_clus',
                                          k = 10)

In [ ]:
cor_clusters <- spat_cor_netw_DT$cor_clusters$spat_netw_clus

In [ ]:
saveRDS(spat_cor_netw_DT,'spat_cor_netw_DT.rds') 

In [ ]:
options(repr.plot.width=28, repr.plot.height=28)
heatmSpatialCorFeats(fov_join, spatCorObject = spat_cor_netw_DT, use_clus_name = 'spat_netw_clus',show_row_names = T)

In [ ]:
library(future)

In [ ]:
options(future.globals.maxSize= 891289600)
## identify genes that are associated with proximity to other cell types
## take extremely long time (about 3days?)
plan('multisession', workers = 1)
ICFsForesHighGenes =  findInteractionChangedFeats(gobject = fov_join,
                              selected_feats = fov_join@feat_ID$rna,
                              spatial_network_name = 'large_network',
                              cluster_column = 'new_clus',
                              diff_test = 'permutation',
                              adjust_method = 'fdr',
                              nr_permutations = 2000,
                              do_parallel = F)

In [ ]:
saveRDS(ICFsForesHighGenes,'ICFsForesHighGenes.rds')

In [ ]:
library(Giotto)

In [ ]:
fov_join <- loadGiotto('saveGiottoDir_final_before_submission/')

In [ ]:
ICFsForesHighGenes <- readRDS('ICFsForesHighGenes.rds')

In [ ]:
ICFsForesHighGenes_stroma_epi <- ICFsForesHighGenes

In [ ]:
ICFsForesHighGenes_epi_stroma <- ICFsForesHighGenes

In [ ]:
ICFsForesHighGenes_epi_epi <- ICFsForesHighGenes

In [ ]:
ICFsForesHighGenes_stroma_stroma <- ICFsForesHighGenes

In [ ]:
ICFsForesHighGenes_epi_stroma$ICFscores <- ICFsForesHighGenes_epi_stroma$ICFscores[
(ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'Chief' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'OLFM4+Epi' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'NE2' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'PG/Neck1' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'Pyloric_deep' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'PG/Neck2' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'NE1' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'ADH1+GKN1-F' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'LEFTY1_epi' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'Gob' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'Ent2' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'PC' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'Foveolar' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'GKN+F' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'Ent1' |
ICFsForesHighGenes_epi_stroma$ICFscores$int_cell_type == 'Ent3') &
(ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'T CD8 memory' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'macrophage' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'T CD4 memory' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'plasma' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'CCL11LFB' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'FibSmo' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'SM' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'T CD8 naive' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'Treg' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'mDC' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'endothelial' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'KLF+FB' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'pericyte' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'T CD4 naive' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'pDC' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'B-cell' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'mast' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'monocyte' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'PDGFR+FB' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'NK' |
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'neutrophil' |                   
ICFsForesHighGenes_epi_stroma$ICFscores$cell_type == 'pro_B-cell')                  
               ]

In [ ]:
ICFsForesHighGenes_stroma_epi$ICFscores <- ICFsForesHighGenes_stroma_epi$ICFscores[
(ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'Chief' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'OLFM4+Epi' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'NE2' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'PG/Neck1' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'Pyloric_deep' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'PG/Neck2' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'NE1' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'ADH1+GKN1-F' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'LEFTY1_epi' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'Gob' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'Ent2' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'PC' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'Foveolar' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'GKN+F' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'Ent1' |
ICFsForesHighGenes_stroma_epi$ICFscores$cell_type == 'Ent3') &
(ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'T CD8 memory' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'macrophage' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'T CD4 memory' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'plasma' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'CCL11LFB' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'FibSmo' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'SM' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'T CD8 naive' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'Treg' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'mDC' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'endothelial' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'KLF+FB' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'pericyte' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'T CD4 naive' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'pDC' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'B-cell' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'mast' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'monocyte' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'PDGFR+FB' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'NK' |
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'neutrophil' |                   
ICFsForesHighGenes_stroma_epi$ICFscores$int_cell_type == 'pro_B-cell')                   
               ]

In [ ]:
ICFsForesHighGenes_epi_epi$ICFscores <- ICFsForesHighGenes_epi_epi$ICFscores[
(ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'Chief' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'OLFM4+Epi' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'NE2' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'PG/Neck1' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'Pyloric_deep' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'PG/Neck2' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'NE1' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'ADH1+GKN1-F' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'LEFTY1_epi' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'Gob' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'Ent2' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'PC' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'Foveolar' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'GKN+F' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'Ent1' |
ICFsForesHighGenes_epi_epi$ICFscores$cell_type == 'Ent3') &
(ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'Chief' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'OLFM4+Epi' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'NE2' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'PG/Neck1' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'Pyloric_deep' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'PG/Neck2' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'NE1' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'ADH1+GKN1-F' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'LEFTY1_epi' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'Gob' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'Ent2' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'PC' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'Foveolar' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'GKN+F' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'Ent1' |
ICFsForesHighGenes_epi_epi$ICFscores$int_cell_type == 'Ent3')               ]

#### 上皮→すべての細胞への影響

In [ ]:
## visualize all genes
options(repr.plot.width=10, repr.plot.height=10)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_epi_stroma,
                       method = 'cell_barplot')

In [ ]:
## visualize all genes
options(repr.plot.width=10, repr.plot.height=10)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_epi_stroma,cell_color_code = color_set,
                       method = 'cell_barplot')

In [ ]:
## visualize all genes
options(repr.plot.width=7, repr.plot.height=7)
pdf('Figure7H.pdf',width = 7, height = 7)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_epi_stroma,#cell_color_code = color_set,
                       method = 'dotplot')
dev.off()

In [ ]:
## visualize all genes
options(repr.plot.width=10, repr.plot.height=10)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_epi_stroma,#cell_color_code = color_set,
                       method = 'dotplot')

#### 間質→すべての細胞への影響

In [ ]:
colors <- readRDS('integrated_data/論文用再検討/distinct_colors.Rds')

In [ ]:
## visualize all genes
options(repr.plot.width=10, repr.plot.height=10)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_stroma_epi,cell_color_code = unlist(colors[15]),
                       method = 'cell_barplot')

In [ ]:
## visualize all genes
options(repr.plot.width=14, repr.plot.height=14)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_stroma_epi,#cell_color_code = color_set,
                       method = 'dotplot')

In [ ]:
## visualize all genes
options(repr.plot.width=7, repr.plot.height=7)
pdf('Figure7I.pdf',width = 7, height = 7)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_stroma_epi,#cell_color_code = color_set,
                       method = 'dotplot')
dev.off()

In [ ]:
## filter genes
ICFscoresFilt = filterICF(ICFsForesHighGenes)

In [ ]:
ICFscoresFilt

In [ ]:
## visualize subset of interaction changed genes (ICFs)
ICF_genes = c('','COL1A1','PDGFRA', 'NRG1','BMP2','BMP4','BMP5','BMP6','WNT5A','WNT7B','WNT2B','WNT3')
ICF_genes_types = c('GKN+F', 'Ent1', 'Ent2', 'Ent3', 'Foveolar','Chief','LEFTY1_epi','PG/Neck1','PG/Neck2','Pyloric_deep','OLFM4+Epi','OLFM4+Epi')
names(ICF_genes) = ICF_genes_types

In [ ]:
ICF_genes

In [ ]:
options(repr.plot.width=14, repr.plot.height=5)
plotICF(gobject = fov_join,
        icfObject = ICFsForesHighGenes,
        source_type = 'PDGFR+FB',
        source_markers = c('COL5A1','COL3A1'),
        ICF_feats = ICF_genes)

In [ ]:
ICFsForesHighGenes_epi_stroma$ICFscores[(ICFsForesHighGenes_epi_stroma$ICFscores$cell_type=='NK') &
                                       (ICFsForesHighGenes_epi_stroma$ICFscores$p.adj<0.05)]$feats[1:100]
     

In [ ]:
## visualize subset of interaction changed genes (ICFs)
ICF_genes = c('WNT7B','THBS2','MIF','ADGRE5','LGALS9','FASLG','CSF1','COL9A2','IL2RG','CD86','')
ICF_genes_types = c('GKN+F', 'Ent1', 'Ent2', 'Ent3', 'Foveolar','Chief','LEFTY1_epi','PG/Neck1','PG/Neck2','Pyloric_deep','OLFM4+Epi')
names(ICF_genes) = ICF_genes_types

In [ ]:
ICF_genes

In [ ]:
options(repr.plot.width=14, repr.plot.height=5)
plotICF(gobject = fov_join,
        icfObject = ICFsForesHighGenes,
        source_type = 'NK',
        source_markers = c('GZMA','GZMB','GNLY'),
        ICF_feats = ICF_genes)

In [ ]:
unique(ICFscoresFilt$ICFscores$feats)[grepl('^BMP',unique(ICFscoresFilt$ICFscores$feats))]

In [ ]:
unique(ICFscoresFilt$ICFscores$feats)[grepl('^WNT',unique(ICFscoresFilt$ICFscores$feats))]

In [ ]:
## visualize all genes
options(repr.plot.width=10, repr.plot.height=10)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_epi_epi,cell_color_code = color_set,
                       method = 'cell_barplot')

In [ ]:
## visualize all genes
options(repr.plot.width=10, repr.plot.height=10)
plotCellProximityFeats(fov_join, icfObject = ICFsForesHighGenes_epi_epi,#cell_color_code = color_set,
                       method = 'dotplot')

In [ ]:
1+1

In [ ]:
saveGiotto(fov_join, foldername = 'saveGiottoDir_final_before_submission',overwrite = TRUE)

# Cellchat

In [ ]:
library(CellChat)

In [ ]:
lefty_id <- pDataDT(fov_join)$cell_ID[grepl('Pyloric-fov001',pDataDT(fov_join)$cell_ID) |
                                     grepl('Pyloric-fov002',pDataDT(fov_join)$cell_ID) |
                                     grepl('Pyloric-fov003',pDataDT(fov_join)$cell_ID) |
                                     grepl('Fundic-fov020',pDataDT(fov_join)$cell_ID) |
                                     grepl('Fundic-fov021',pDataDT(fov_join)$cell_ID) ]

In [ ]:
length(lefty_id)

In [ ]:
#field1_ids = cell_metadata[FOV == 0]$cell_ID
subfov_join = subsetGiotto(fov_join, cell_ids = lefty_id)

In [ ]:
cols <- readRDS('integrated_data/論文用再検討/distinct_colors.Rds')

In [ ]:
data.input = get_expression_values(subfov_join)

In [ ]:
data.input = data.input@exprMat

In [ ]:
dim(data.input)

In [ ]:
meta = pDataDT(subfov_join)

In [ ]:
dim(meta)

In [ ]:
spatial.locs = subfov_join@spatial_locs$cell$raw

In [ ]:
head(meta)

In [ ]:
rownames(meta) <- meta$cell_ID

In [ ]:
scale.factors = list(spot.diameter = 50, spot = 50)

In [ ]:
scale.factors

In [ ]:
spatial.locs <- spatial.locs@coordinates

In [ ]:
spatial.locs <- as.data.frame(spatial.locs)

In [ ]:
rownames(spatial.locs) <- spatial.locs$cell_ID

In [ ]:
spatial.locs <- spatial.locs[,1:2]

In [ ]:
head(spatial.locs)

In [ ]:
cellchat <- createCellChat(object = data.input, meta = meta, group.by = 'new_clus',
                           datatype='spatial', coordinates = spatial.locs, scale.factors = scale.factors)

In [ ]:
CellChatDB <- CellChatDB.human # use CellChatDB.human if running on human data

# use a subset of CellChatDB for cell-cell communication analysis
#CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling") # use Secreted Signaling
# use all CellChatDB for cell-cell communication analysis
# CellChatDB.use <- CellChatDB # simply use the default CellChatDB

# set the used database in the object
cellchat@DB <- CellChatDB

In [ ]:
options(stringsAsFactors = FALSE)
interaction_input <- read.csv(file = './cellchat_database/interaction_input_CellChatDB_new.csv', row.names = 1)
#complex_input <- read.csv(file = './cellchat_database/complex_input_new.csv', row.names = 1)
#cofactor_input <- read.csv(file = './cellchat_database/cofactor_input_new.csv', row.names = 1)
geneInfo <- read.csv(file = './cellchat_database/geneInfo_input_CellChatDB.csv', row.names = 1)

In [ ]:
DB <- CellChatDB.human

In [ ]:
CellChatDB <- list()
CellChatDB$interaction <- interaction_input
CellChatDB$complex <- DB$complex
CellChatDB$cofactor <- DB$cofactor
CellChatDB$geneInfo <- geneInfo

In [ ]:
CellChatDB.use <- CellChatDB # use Secreted Signaling
# use all CellChatDB for cell-cell communication analysis
# CellChatDB.use <- CellChatDB # simply use the default CellChatDB

# set the used database in the object
cellchat@DB <- CellChatDB.use

In [ ]:
# subset the expression data of signaling genes for saving computation cost
cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database
future::plan("multiprocess", workers = 1) # do parallel
#> Warning: Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead,
#> explicitly specify either 'multisession' or 'multicore'. In the current R
#> session, 'multiprocess' equals 'multisession'.
#> Warning in supportsMulticoreAndRStudio(...): [ONE-TIME WARNING] Forked
#> processing ('multicore') is not supported when running R from RStudio
#> because it is considered unstable. For more details, how to control forked
#> processing or not, and how to silence this warning in future R sessions, see ?
#> parallelly::supportsMulticore
cellchat <- suppressWarnings(identifyOverExpressedGenes(cellchat))
cellchat <- suppressWarnings(identifyOverExpressedInteractions(cellchat))
 
# project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` in the function `computeCommunProb()` in order to use the projected data)
# cellchat <- projectData(cellchat, PPI.mouse)

In [ ]:
cellchat <- suppressWarnings(computeCommunProb(cellchat, type = "truncatedMean", trim = 0.1, 
                               distance.use = TRUE, interaction.length = 200, scale.distance = 0.1))
#> truncatedMean is used for calculating the average gene expression per cell group. 
#> [1] ">>> Run CellChat on spatial imaging data using distances as constraints <<< [2022-11-12 07:49:23]"
#> The suggested minimum value of scaled distances is in [1,2], and the calculated value here is  1.30553 
#> [1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2022-11-12 08:10:42]"

# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat <- suppressWarnings(filterCommunication(cellchat, min.cells = 10))

In [ ]:
cellchat <- computeCommunProbPathway(cellchat)

In [ ]:
cellchat <- aggregateNet(cellchat)

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
groupSize <- as.numeric(table(cellchat@idents))
par(mfrow = c(1,2), xpd=TRUE)
netVisual_circle(cellchat@net$count, vertex.weight = rowSums(cellchat@net$count), weight.scale = T, label.edge= F, title.name = "Number of interactions")
netVisual_circle(cellchat@net$weight, vertex.weight = rowSums(cellchat@net$weight), weight.scale = T, label.edge= F, title.name = "Interaction weights/strength")

In [ ]:
# Compute the network centrality scores
cellchat <- netAnalysis_computeCentrality(cellchat, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
pathways.show <- c("NRG") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
# Visualize the computed centrality scores using heatmap, allowing ready identification of major signaling roles of cell groups
par(mfrow=c(1,1))
netAnalysis_signalingRole_network(cellchat, signaling = pathways.show, width = 8, height = 2.5, font.size = 10)

In [ ]:
saveRDS(cellchat, file = "cellchat_cosmx_lefty1.rds")

cellchat <- readRDS('cellchat_cosmx.rds')

In [ ]:
mat <- cellchat@net$weight
par(mfrow = c(3,4), xpd=TRUE)
for (i in 1:nrow(mat)) {
  mat2 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat2[i, ] <- mat[i, ]
  netVisual_circle(mat2, vertex.weight = groupSize, weight.scale = T, edge.weight.max = max(mat), title.name = rownames(mat)[i])
}

In [ ]:
library(ComplexHeatmap)

In [ ]:
options(repr.plot.width=14,repr.plot.height=14)
# show all the significant interactions (L-R pairs) from some cell groups (defined by 'sources.use') to other cell groups (defined by 'targets.use')
netVisual_bubble(cellchat,remove.isolate = FALSE, targets.use = 'LEFTY1_epi',)
#> Comparing communications on a single object

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("NRG") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("WNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("ncWNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("BMP") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("TGFb") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("EPHB") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("NOTCH") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
# Visualize the computed centrality scores using heatmap, allowing ready identification of major signaling roles of cell groups
par(mfrow=c(1,1))
netAnalysis_signalingRole_network(cellchat, signaling = pathways.show, width = 8, height = 2.5, font.size = 10)

In [ ]:
# Spatial plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "spatial", edge.width.max = 2, vertex.size.max = 1, alpha.image = 0.2, vertex.label.cex = 3.5)

In [ ]:
cellchat <- netAnalysis_computeCentrality(cellchat, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht1 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing",width = 24,height = 24)

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht2 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming",width = 24,height = 24)

In [ ]:
ht1

In [ ]:
ht2

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat, sources.use = c('NK'), targets.use = c('GKN+F','Foveolar','TA_Ent','Ent2','LEFTY1_epi'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=10,repr.plot.height=10)
netVisual_bubble(cellchat, targets.use = c('OLFM4+Epi'),  remove.isolate = TRUE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=10,repr.plot.height=10)
netVisual_bubble(cellchat, targets.use = c('epi_ithmus'),  remove.isolate = TRUE)
#> Comparing communications on a single object

In [ ]:
unique(cellchat@DB$interaction$pathway_name)

In [ ]:
cellchat@DB$interaction[cellchat@DB$interaction$pathway_name == 'TWEAK',]

In [ ]:
cellchat@DB$interaction[cellchat@DB$interaction$pathway_name == 'WNT',]

In [ ]:
cellchat@DB$interaction[cellchat@DB$interaction$pathway_name == 'ncWNT',]

In [ ]:
pathways.show <- c("COLLAGEN") 
# Circle plot
options(repr.plot.width=14,repr.plot.height=14)
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
pathways.show <- c("PDGF") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
lefty_id <- pDataDT(fov_join)$cell_ID[grepl('Pyloric-fov001',pDataDT(fov_join)$cell_ID) |
                                     grepl('Pyloric-fov002',pDataDT(fov_join)$cell_ID) |
                                     grepl('Pyloric-fov003',pDataDT(fov_join)$cell_ID) |
                                     grepl('Fundic-fov020',pDataDT(fov_join)$cell_ID) |
                                     grepl('Fundic-fov021',pDataDT(fov_join)$cell_ID) ]

### 部位別cellchat

In [ ]:
unique(pDataDT(fov_join)$Tissue_spatial_region)

In [ ]:
unique(pDataDT(fov_join)$Tissue_spatial_region)[grepl('Metaplasia',unique(pDataDT(fov_join)$Tissue_spatial_region))]

In [ ]:
pyloric <- pDataDT(fov_join)$cell_ID[(pDataDT(fov_join)$Tissue_spatial_region=='Pyloric_deep') |
                                    (pDataDT(fov_join)$Tissue_spatial_region=='Pyloric_ithmus') |
                                    (pDataDT(fov_join)$Tissue_spatial_region=='Pyloric_superficial')
                                    ]

In [ ]:
fundic <- pDataDT(fov_join)$cell_ID[(pDataDT(fov_join)$Tissue_spatial_region=='Fundic_deep') |
                                    (pDataDT(fov_join)$Tissue_spatial_region=='Fundic_ithmus') |
                                    (pDataDT(fov_join)$Tissue_spatial_region=='Fundic_superficial')
                                    ]

In [ ]:
IM <- pDataDT(fov_join)$cell_ID[(pDataDT(fov_join)$Tissue_spatial_region=='Metaplasia_deep') |
                                    (pDataDT(fov_join)$Tissue_spatial_region=='Metaplasia_superficial')
                                    ]

In [ ]:
length(pyloric)

In [ ]:
length(fundic)

In [ ]:
length(IM)

In [ ]:
#field1_ids = cell_metadata[FOV == 0]$cell_ID
subfov_join_pyloric = subsetGiotto(fov_join, cell_ids = pyloric)

In [ ]:
cols <- readRDS('integrated_data/論文用再検討/distinct_colors.Rds')

In [ ]:
data.input = get_expression_values(subfov_join_pyloric)

In [ ]:
data.input = data.input@exprMat

In [ ]:
dim(data.input)

In [ ]:
meta = pDataDT(subfov_join_pyloric)

In [ ]:
dim(meta)

In [ ]:
spatial.locs = subfov_join_pyloric@spatial_locs$cell$raw

In [ ]:
head(meta)

In [ ]:
rownames(meta) <- meta$cell_ID

In [ ]:
scale.factors = list(spot.diameter = 50, spot = 50)

In [ ]:
scale.factors

In [ ]:
spatial.locs <- spatial.locs@coordinates

In [ ]:
spatial.locs <- as.data.frame(spatial.locs)

In [ ]:
rownames(spatial.locs) <- spatial.locs$cell_ID

In [ ]:
spatial.locs <- spatial.locs[,1:2]

In [ ]:
head(spatial.locs)

In [ ]:
cellchat_pyloric <- createCellChat(object = data.input, meta = meta, group.by = 'new_clus',
                           datatype='spatial', coordinates = spatial.locs, scale.factors = scale.factors)

In [ ]:
# set the used database in the object
cellchat_pyloric@DB <- CellChatDB.use

In [ ]:
# subset the expression data of signaling genes for saving computation cost
cellchat_pyloric <- subsetData(cellchat_pyloric) # This step is necessary even if using the whole database
future::plan("multiprocess", workers = 1) # do parallel
#> Warning: Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead,
#> explicitly specify either 'multisession' or 'multicore'. In the current R
#> session, 'multiprocess' equals 'multisession'.
#> Warning in supportsMulticoreAndRStudio(...): [ONE-TIME WARNING] Forked
#> processing ('multicore') is not supported when running R from RStudio
#> because it is considered unstable. For more details, how to control forked
#> processing or not, and how to silence this warning in future R sessions, see ?
#> parallelly::supportsMulticore
cellchat_pyloric <- suppressWarnings(identifyOverExpressedGenes(cellchat_pyloric))
cellchat_pyloric <- suppressWarnings(identifyOverExpressedInteractions(cellchat_pyloric))
 
# project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` in the function `computeCommunProb()` in order to use the projected data)
# cellchat <- projectData(cellchat, PPI.mouse)

In [ ]:
cellchat_pyloric <- suppressWarnings(computeCommunProb(cellchat_pyloric, type = "truncatedMean", trim = 0.1, 
                               distance.use = TRUE, interaction.length = 200, scale.distance = 0.1))
#> truncatedMean is used for calculating the average gene expression per cell group. 
#> [1] ">>> Run CellChat on spatial imaging data using distances as constraints <<< [2022-11-12 07:49:23]"
#> The suggested minimum value of scaled distances is in [1,2], and the calculated value here is  1.30553 
#> [1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2022-11-12 08:10:42]"

# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat_pyloric <- suppressWarnings(filterCommunication(cellchat_pyloric, min.cells = 10))

In [ ]:
cellchat_pyloric <- computeCommunProbPathway(cellchat_pyloric)

In [ ]:
cellchat_pyloric <- aggregateNet(cellchat_pyloric)

In [ ]:
1+1

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
groupSize <- as.numeric(table(cellchat_pyloric@idents))
par(mfrow = c(1,2), xpd=TRUE)
netVisual_circle(cellchat_pyloric@net$count, vertex.weight = rowSums(cellchat_pyloric@net$count), weight.scale = T, label.edge= F, title.name = "Number of interactions")
netVisual_circle(cellchat_pyloric@net$weight, vertex.weight = rowSums(cellchat_pyloric@net$weight), weight.scale = T, label.edge= F, title.name = "Interaction weights/strength")

In [ ]:
# Compute the network centrality scores
cellchat_pyloric <- netAnalysis_computeCentrality(cellchat_pyloric, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
pathways.show <- c("NRG") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
saveRDS(cellchat_pyloric, file = "cellchat_cosmx_pyloric.rds")

cellchat <- readRDS('cellchat_cosmx.rds')

In [ ]:
mat <- cellchat_pyloric@net$weight
par(mfrow = c(3,4), xpd=TRUE)
for (i in 1:nrow(mat)) {
  mat2 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat2[i, ] <- mat[i, ]
  netVisual_circle(mat2, vertex.weight = groupSize, weight.scale = T, edge.weight.max = max(mat), title.name = rownames(mat)[i])
}

In [ ]:
options(repr.plot.width=14,repr.plot.height=14)
# show all the significant interactions (L-R pairs) from some cell groups (defined by 'sources.use') to other cell groups (defined by 'targets.use')
netVisual_bubble(cellchat_pyloric,remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("NRG") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("WNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("ncWNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("BMP") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("TGFb") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("EPHB") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("NOTCH") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_pyloric, signaling = pathways.show, layout = "circle")

In [ ]:
cellchat <- netAnalysis_computeCentrality(cellchat_pyloric, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht1 <- netAnalysis_signalingRole_heatmap(cellchat_pyloric, pattern = "outgoing",width = 24,height = 24)

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht2 <- netAnalysis_signalingRole_heatmap(cellchat_pyloric, pattern = "incoming",width = 24,height = 24)

In [ ]:
ht1

In [ ]:
ht2

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_pyloric, sources.use = c('NK'), targets.use = c('GKN+F','Foveolar','TA_Ent','Ent2','LEFTY1_epi'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=10,repr.plot.height=10)
netVisual_bubble(cellchat_pyloric, targets.use = c('OLFM4+Epi'),  remove.isolate = TRUE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=10,repr.plot.height=10)
netVisual_bubble(cellchat, targets.use = c('epi_ithmus'),  remove.isolate = TRUE)
#> Comparing communications on a single object

In [ ]:
unique(cellchat@DB$interaction$pathway_name)

In [ ]:
cellchat@DB$interaction[cellchat@DB$interaction$pathway_name == 'TWEAK',]

In [ ]:
cellchat@DB$interaction[cellchat@DB$interaction$pathway_name == 'WNT',]

In [ ]:
cellchat@DB$interaction[cellchat@DB$interaction$pathway_name == 'ncWNT',]

In [ ]:
pathways.show <- c("COLLAGEN") 
# Circle plot
options(repr.plot.width=14,repr.plot.height=14)
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
pathways.show <- c("PDGF") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle")

In [ ]:
lefty_id <- pDataDT(fov_join)$cell_ID[grepl('Pyloric-fov001',pDataDT(fov_join)$cell_ID) |
                                     grepl('Pyloric-fov002',pDataDT(fov_join)$cell_ID) |
                                     grepl('Pyloric-fov003',pDataDT(fov_join)$cell_ID) |
                                     grepl('Fundic-fov020',pDataDT(fov_join)$cell_ID) |
                                     grepl('Fundic-fov021',pDataDT(fov_join)$cell_ID) ]

# all dataset cellchat

#### cellchat準備

In [ ]:
length(unique(pDataDT(fov_join)$new_clus))

In [ ]:
library(Giotto)

In [ ]:
fov_join <- loadGiotto('saveGiottoDir_final_before_submission')

In [ ]:
cluster_order = unique(fov_join@cell_metadata$cell$rna$cell_types)

In [ ]:
cluster_order = sort(cluster_order)

In [ ]:
## add cell types ###
cluster_order = sort(cluster_order)
clusters_cell_types_lung = c('TA_Ent', 'ADH1+GKN1-F', 'pro_B-cell', 'B-cell',#a,ADH1+F,b,B-cell
                             'Chief', 'CCL11+FB','Pyloric_deep','LEFTY1_epi', #c, CCL11+FB, chief, d
                             'OLFM4+Epi','endothelial','Ent','Stromal', #e,endothelial,Ent,f
                             'FibSmo','Ent2', #'FibSmo','g'
                             'GKN+F', 'Gob', 'plasma', 'plasma', #'GKN+F', 'Gob', 'h', 'i'
                             'Foveolar','KLF+FB',#j, KLF+FB
                             "macrophage","mast","mDC","pericyte",#"macrophage","mast","mDC","MFB"
                             'monocyte','NE1','NE2','neutrophil',
                             'NK','PC','pDC','PDGFR+FB','PG/Neck1', 'PG/Neck2', 'plasma','pericyte_SM','T CD4 memory', 'T CD4 naive', 'T CD8 memory','T CD8 naive','Treg')

names(clusters_cell_types_lung) = as.character(sort(cluster_order))

In [ ]:
trans <- data.frame(
    old = names(clusters_cell_types_lung),
    new = clusters_cell_types_lung
    )

In [ ]:
new_clus <- lapply(pDataDT(fov_join)$cell_types, function(x) {
    as.vector(trans$new[match(x, trans$old)])
    }
       )

In [ ]:
new_clus <- unlist(new_clus)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = new_clus)

In [ ]:
unique(pDataDT(fov_join)$new_clus)

In [ ]:
#Set desired cell order
cell.levels <- c("GKN+F","Foveolar", "ADH1+GKN1-F","PG/Neck1","PG/Neck2", "Pyloric_deep","Chief", "PC",
                "Ent", "Ent2",'Gob',"TA_Ent", "OLFM4+Epi",'LEFTY1_epi','NE1','NE2',
                'T CD8 memory','T CD8 naive','T CD4 memory','T CD4 naive', "Treg",'NK',
                 'B-cell','pro_B-cell','plasma',
                "KLF+FB","PDGFR+FB", 'pericyte',"pericyte_SM",
                "CCL11+FB","FibSmo", "monocyte","macrophage",'neutrophil',"mDC","pDC",
                "mast", "endothelial",'Stromal'
                )

In [ ]:
library(CellChat)

In [ ]:
mucosa_ids <- pDataDT(fov_join)$cell_ID[
    (pDataDT(fov_join)$Tissue_spatial_region != 'lymphoid follicule') &
    (pDataDT(fov_join)$Tissue_spatial_region != 'Submucosa_deep') &
    (pDataDT(fov_join)$Tissue_spatial_region != 'Muscle_layer')
]

In [ ]:
length(mucosa_ids)

In [ ]:
ls()

In [ ]:
remove(smallfov,smallfov_sub,smallfov2,counts_fundic, counts_feat, counts_neg, counts_pyloric)

In [ ]:
remove(cosmx, count_replace, counts, counts_all, umaps)

In [ ]:
fov_join_mucosa <- subsetGiotto(fov_join, cell_ids = mucosa_ids)

In [ ]:
data.input = get_expression_values(fov_join_mucosa)

In [ ]:
data.input = data.input@exprMat

In [ ]:
dim(data.input)

In [ ]:
meta = pDataDT(fov_join_mucosa)

In [ ]:
dim(meta)

In [ ]:
spatial.locs = fov_join_mucosa@spatial_locs$cell$raw

In [ ]:
rownames(meta) <- meta$cell_ID

In [ ]:
head(meta)

In [ ]:
colnames(meta)

In [ ]:
min(meta$Area)*0.18*0.18

In [ ]:
min(meta$Height)

In [ ]:
scale.factors = list(spot.diameter = 6.0264, spot = 6.0264)

In [ ]:
scale.factors

In [ ]:
spatial.locs <- spatial.locs@coordinates

In [ ]:
spatial.locs <- as.data.frame(spatial.locs)

In [ ]:
rownames(spatial.locs) <- spatial.locs$cell_ID

In [ ]:
spatial.locs <- spatial.locs[,1:2]

In [ ]:
head(spatial.locs)

In [ ]:
length(unique(meta$new_clus))

In [ ]:
length(cell.levels)

In [ ]:
cellchat_all <- createCellChat(object = data.input, meta = meta, group.by = 'new_clus',
                           datatype='spatial', coordinates = spatial.locs, scale.factors = scale.factors)

In [ ]:
options(stringsAsFactors = FALSE)
interaction_input <- read.csv(file = './cellchat_database/interaction_input_CellChatDB_new.csv', row.names = 1)
#complex_input <- read.csv(file = './cellchat_database/complex_input_new.csv', row.names = 1)
#cofactor_input <- read.csv(file = './cellchat_database/cofactor_input_new.csv', row.names = 1)
geneInfo <- read.csv(file = './cellchat_database/geneInfo_input_CellChatDB.csv', row.names = 1)

In [ ]:
DB <- CellChatDB.human

In [ ]:
CellChatDB <- list()
CellChatDB$interaction <- interaction_input
CellChatDB$complex <- DB$complex
CellChatDB$cofactor <- DB$cofactor
CellChatDB$geneInfo <- geneInfo

In [ ]:
CellChatDB.use <- CellChatDB # use Secreted Signaling
# use all CellChatDB for cell-cell communication analysis
# CellChatDB.use <- CellChatDB # simply use the default CellChatDB

# set the used database in the object
cellchat_all@DB <- CellChatDB.use

cellchat_all <- setIdent(cellchat_all, ident.use = "new_clus", levels = cell.levels)

In [ ]:
cellchat_all@idents <- factor(cellchat_all@idents, levels=cell.levels)

In [ ]:
unique(cellchat_all@idents)

In [ ]:
# subset the expression data of signaling genes for saving computation cost
cellchat_all <- subsetData(cellchat_all) # This step is necessary even if using the whole database
future::plan("multiprocess", workers = 1) # do parallel
#> Warning: Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead,
#> explicitly specify either 'multisession' or 'multicore'. In the current R
#> session, 'multiprocess' equals 'multisession'.
#> Warning in supportsMulticoreAndRStudio(...): [ONE-TIME WARNING] Forked
#> processing ('multicore') is not supported when running R from RStudio
#> because it is considered unstable. For more details, how to control forked
#> processing or not, and how to silence this warning in future R sessions, see ?
#> parallelly::supportsMulticore
cellchat_all <- suppressWarnings(identifyOverExpressedGenes(cellchat_all))
cellchat_all <- suppressWarnings(identifyOverExpressedInteractions(cellchat_all))
 
# project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` in the function `computeCommunProb()` in order to use the projected data)
# cellchat <- projectData(cellchat, PPI.mouse)

In [ ]:
cellchat_all <- suppressWarnings(computeCommunProb(cellchat_all, type = "truncatedMean", trim = 0.1, 
                               distance.use = TRUE, interaction.length = 200, scale.distance = 0.1))

In [ ]:
1

In [ ]:

#> truncatedMean is used for calculating the average gene expression per cell group. 
#> [1] ">>> Run CellChat on spatial imaging data using distances as constraints <<< [2022-11-12 07:49:23]"
#> The suggested minimum value of scaled distances is in [1,2], and the calculated value here is  1.30553 
#> [1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2022-11-12 08:10:42]"

# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat_all <- suppressWarnings(filterCommunication(cellchat_all, min.cells = 10))

In [ ]:
cellchat_all <- computeCommunProbPathway(cellchat_all)

In [ ]:
cellchat_all <- aggregateNet(cellchat_all)

In [ ]:
cellchat_all

In [ ]:
head(cellchat_all@images)

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
groupSize <- as.numeric(table(cellchat_all@idents))
par(mfrow = c(1,2), xpd=TRUE)
netVisual_circle(cellchat_all@net$count, vertex.weight = rowSums(cellchat_all@net$count), weight.scale = T, 
                 label.edge= F, title.name = "Number of interactions",vertex.label.cex = 0.45)
netVisual_circle(cellchat_all@net$weight, vertex.weight = rowSums(cellchat_all@net$weight),
                 weight.scale = T, label.edge= F, title.name = "Interaction weights/strength",
                vertex.label.cex = 0.45)

In [ ]:
# Compute the network centrality scores
cellchat_all <- netAnalysis_computeCentrality(cellchat_all, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
saveRDS(cellchat_all, file = "cellchat_cosmx_all.rds")

# all dataset cellchat, replacing expression matrix to scRNAseq

#### cellchat準備

#### calculate the representative gene expression value for each cluster

In [ ]:
library(Seurat)

In [ ]:
epi <- readRDS('./for_analysis/論文用再検討/修正/data_9_99_monocle3_seurat_epi.rds')

In [ ]:
fb <- readRDS('./for_analysis/論文用再検討/修正/data_9_3_annotated_fibroblasts.rds')

In [ ]:
mp <- readRDS('./for_analysis/論文用再検討/修正/data_9_2_annotated_macrophage.rds')

In [ ]:
mp <- subset(mp, idents='monocyte')

In [ ]:
unique(Idents(epi))

# 上皮

In [ ]:
Chief <- subset(epi, idents = 'Chief')

In [ ]:
PGneck1 <- subset(epi, idents = 'PG/Neck1')

In [ ]:
PGneck2 <- subset(epi, idents = 'PG/Neck2')

In [ ]:
GKNF <- subset(epi, idents = 'GKN+F')

In [ ]:
PC <- subset(epi, idents = 'PC')

In [ ]:
Pr_epi <- subset(epi, idents = 'Pr_epi')

In [ ]:
ADH1GKN1F <- subset(epi, idents = 'ADH1+GKN1-F')

In [ ]:
NE2 <- subset(epi, idents = 'NE2')

In [ ]:
NE1 <- subset(epi, idents = 'NE1')

In [ ]:
Ent <- subset(epi, idents = 'Ent')

In [ ]:
Gob <- subset(epi, idents = 'Gob')

In [ ]:
LEFTY1ADHF <- subset(epi, idents = 'LEFTY1+ADH+F')

In [ ]:
LEFTY1pgneck <- subset(epi, idents = 'LEFTY1+PG/Neck')

In [ ]:
EM <- function(vector){
    return(as.numeric(1/2 * vector[3] + 1/4 * (vector[1] + vector[4])))
    }

In [ ]:
calc_EM <- function(cluster){
    x <- apply(cluster@assays$RNA@counts,1,quantile)
    y <- t(apply(x,2,EM))
    rownames(y) <- deparse(substitute(cluster))
    return (t(y))
    }

In [ ]:
z <- calc_EM(Chief)

In [ ]:
cluster_name <- c('Chief','PGneck1','PGneck2','GKNF','PC',
                 'Pr_epi','ADH1GKN1F','NE2','NE1','Ent','Gob','LEFTY1ADHF','LEFTY1pgneck')

In [ ]:
for (i in 2:length(cluster_name)){
    z <- cbind(z, calc_EM(get(cluster_name[i])))
    colnames(z)[i] <- cluster_name[i]
    }

In [ ]:
z4 <- calc_EM(mp)

In [ ]:
colnames(z4) <- 'macrophage'

# Fibro

In [ ]:
unique(Idents(fb))

In [ ]:
FB.data <- fb@assays$RNA@counts

In [ ]:
PDGFR <- subset(fb, idents = 'PDGFR+FB')

In [ ]:
CCL11 <- subset(fb, idents = 'CCL11+FB')

In [ ]:
FibSmo <- subset(fb, idents = 'FibSmo')

In [ ]:
z2 <- calc_EM(PDGFR)

In [ ]:
cluster_name2 <- c('PDGFR','CCL11','FibSmo')

In [ ]:
for (i in 2:length(cluster_name2)){
    z2 <- cbind(z2, calc_EM(get(cluster_name2[i])))
    colnames(z2)[i] <- cluster_name2[i]
    }

In [ ]:
colnames(z2) <- c('PDGFR+FB','CCL11+FB','FibSmo')

In [ ]:
head(z2)

In [ ]:
head(z)

In [ ]:
unique(Idents(epi))

In [ ]:
colnames(z) <- c('Chief','PG/Neck1','PG/Neck2','GKN+F','PC',
                 'Pr_epi','ADH1+GKN1-F','NE2','NE1','Ent','Gob','LEFTY1+ADH+F','LEFTY1+PG/Neck')

In [ ]:
library(Giotto)

In [ ]:
fov_join <- loadGiotto('saveGiottoDir_final_before_submission')

In [ ]:
cluster_order = unique(fov_join@cell_metadata$cell$rna$cell_types)

In [ ]:
cluster_order = sort(cluster_order)

In [ ]:
unique(cluster_order)

In [ ]:
## add cell types ###
cluster_order = sort(cluster_order)
clusters_cell_types_lung = c('Ent', 'ADH1+GKN1-F', 'pro_B-cell', 'B-cell',#a,ADH1+F,b,B-cell
                             'Chief', 'CCL11+FB','PG/Neck2','LEFTY1+PG/Neck', #c, CCL11+FB, chief, d
                             'LEFTY1+ADH+F','endothelial','Ent','Stromal', #e,endothelial,Ent,f
                             'FibSmo','Ent', #'FibSmo','g'
                             'GKN+F', 'Gob', 'plasma', 'plasma', #'GKN+F', 'Gob', 'h', 'i'
                             'GKN+F','KLF+FB',#j, KLF+FB
                             "macrophage","mast","mDC","pericyte",#"macrophage","mast","mDC","MFB"
                             'monocyte','NE1','NE2','neutrophil',
                             'NK','PC','pDC','PDGFR+FB','PG/Neck1', 'PG/Neck2', 'plasma','pericyte_SM','T CD4 memory', 'T CD4 naive', 'T CD8 memory','T CD8 naive','Treg')

names(clusters_cell_types_lung) = as.character(sort(cluster_order))

In [ ]:
trans <- data.frame(
    old = names(clusters_cell_types_lung),
    new = clusters_cell_types_lung
    )

In [ ]:
new_clus <- lapply(pDataDT(fov_join)$cell_types, function(x) {
    as.vector(trans$new[match(x, trans$old)])
    }
       )

In [ ]:
new_clus <- unlist(new_clus)

In [ ]:
fov_join <- addCellMetadata(fov_join, new_metadata = new_clus)

In [ ]:
unique(pDataDT(fov_join)$new_clus)

In [ ]:
#Set desired cell order
cell.levels <- c("GKN+F", "ADH1+GKN1-F","PG/Neck1","PG/Neck2","Chief", "PC",
                "Ent",'Gob',"LEFTY1+ADH+F",'LEFTY1+PG/Neck','NE1','NE2',
                 "PDGFR+FB","CCL11+FB","FibSmo",'macrophage'
                )

In [ ]:
epi_fb <- pDataDT(fov_join)$cell_ID[
    (pDataDT(fov_join)$new_clus == 'GKN+F') |
    (pDataDT(fov_join)$new_clus == 'ADH1+GKN1-F') |
    (pDataDT(fov_join)$new_clus == 'PG/Neck1') |
    (pDataDT(fov_join)$new_clus == 'PG/Neck2') |
        (pDataDT(fov_join)$new_clus == 'Chief') |
        (pDataDT(fov_join)$new_clus == 'PC') |
        (pDataDT(fov_join)$new_clus == 'Ent') |
        (pDataDT(fov_join)$new_clus == 'Gob') |
        (pDataDT(fov_join)$new_clus == 'LEFTY1+ADH+F') |
        (pDataDT(fov_join)$new_clus == 'LEFTY1+PG/Neck') |
        (pDataDT(fov_join)$new_clus == 'NE1') |
            (pDataDT(fov_join)$new_clus == 'NE2') |
            (pDataDT(fov_join)$new_clus == 'PDGFR+FB') |
            (pDataDT(fov_join)$new_clus == 'CCL11+FB') |
            (pDataDT(fov_join)$new_clus == 'FibSmo') |
                (pDataDT(fov_join)$new_clus == 'macrophage')
]

In [ ]:
fov_join <- subsetGiotto(fov_join, cell_ids = epi_fb)

In [ ]:
library(CellChat)

In [ ]:
mucosa_ids <- pDataDT(fov_join)$cell_ID[
    (pDataDT(fov_join)$Tissue_spatial_region != 'lymphoid follicule') &
    (pDataDT(fov_join)$Tissue_spatial_region != 'Submucosa_deep') &
    (pDataDT(fov_join)$Tissue_spatial_region != 'Muscle_layer')
]

In [ ]:
length(mucosa_ids)

In [ ]:
ls()

In [ ]:
remove(smallfov,smallfov_sub,smallfov2,counts_fundic, counts_feat, counts_neg, counts_pyloric)

In [ ]:
remove(cosmx, count_replace, counts, counts_all, umaps)

In [ ]:
fov_join_mucosa <- subsetGiotto(fov_join, cell_ids = mucosa_ids)

In [ ]:
z3 <- cbind(z,z2,z4)

In [ ]:
head(z3)

In [ ]:
data.input = get_expression_values(fov_join_mucosa)

In [ ]:
data.input = data.input@exprMat

In [ ]:
dim(data.input)

In [ ]:
meta = pDataDT(fov_join_mucosa)

In [ ]:
dim(meta)

In [ ]:
spatial.locs = fov_join_mucosa@spatial_locs$cell$raw

In [ ]:
rownames(meta) <- meta$cell_ID

In [ ]:
head(meta)

In [ ]:
colnames(meta)

In [ ]:
min(meta$Area)*0.18*0.18

In [ ]:
min(meta$Height)

In [ ]:
scale.factors = list(spot.diameter = 6.0264, spot = 6.0264)

In [ ]:
scale.factors

In [ ]:
spatial.locs <- spatial.locs@coordinates

In [ ]:
spatial.locs <- as.data.frame(spatial.locs)

In [ ]:
rownames(spatial.locs) <- spatial.locs$cell_ID

In [ ]:
spatial.locs <- spatial.locs[,1:2]

In [ ]:
head(spatial.locs)

In [ ]:
length(unique(meta$new_clus))

In [ ]:
length(cell.levels)

In [ ]:
cell.levels

#### data.inputの発現行列をseuratの代表値に置き換えたい

In [ ]:
head(data.input)

In [ ]:
head(colnames(data.input))

In [ ]:
# Create dictionary of cellType by cellID
cell_dict <- as.list(meta$new_clus)
names(cell_dict) <- meta$cell_ID

In [ ]:
unique(cell_dict)

In [ ]:
z3 <- as.data.frame(z3)

test <- matrix(rep(z3$`CCL11+FB`, times=dim(data.input[,colnames(data.input) %in% names(cell_dict)[cell_dict == "CCL11+FB"]])[2]), 
       nrow=dim(data.input[,colnames(data.input) %in% names(cell_dict)[cell_dict == "CCL11+FB"]])[2],
       ncol=length(z3$`CCL11+FB`), byrow=TRUE)

head(colnames(data.input[,colnames(data.input) %in% names(cell_dict)[cell_dict == "CCL11+FB"]]))

rownames(test) <- colnames(data.input[,colnames(data.input) %in% names(cell_dict)[cell_dict == "CCL11+FB"]])

colnames(test) <- rownames(z3)

In [ ]:
# Create an empty list to store the test matrices
test_list <- c()

# Loop through the column names of z3
for(col_name in colnames(z3)) {
  # Get the corresponding cell type from cell_dict
  cell_type <- cell_dict[col_name]
  
  # Get the subset of data.input that corresponds to the current cell type
  data_subset <- data.input[,colnames(data.input) %in% names(cell_dict)[cell_dict == col_name]]
  
  # Create the test matrix for the current column of z3
  test <- matrix(rep(z3[,col_name], times=dim(data_subset)[2]), 
                 nrow=dim(data_subset)[2],
                 ncol=length(z3[,col_name]), byrow=TRUE)
  
  # Set the row names and column names of the test matrix
  rownames(test) <- colnames(data_subset)
  colnames(test) <- rownames(z3)
  
  # Add the test matrix to the test_list
  test_list <- rbind(test_list, test)
}

In [ ]:
head(colnames(data.input))

In [ ]:
test_list <- t(test_list)

In [ ]:
test_list <- test_list[,colnames(data.input)]

In [ ]:
library(Matrix)

In [ ]:
test_list <- Matrix(test_list, sparse=TRUE)

In [ ]:
head(test_list)

In [ ]:
head(data.input)

In [ ]:
cellchat_all <- createCellChat(object = test_list, meta = meta, group.by = 'new_clus',
                           datatype='spatial', coordinates = spatial.locs, scale.factors = scale.factors)

In [ ]:
options(stringsAsFactors = FALSE)
interaction_input <- read.csv(file = './cellchat_database/interaction_input_CellChatDB_new.csv', row.names = 1)
#complex_input <- read.csv(file = './cellchat_database/complex_input_new.csv', row.names = 1)
#cofactor_input <- read.csv(file = './cellchat_database/cofactor_input_new.csv', row.names = 1)
geneInfo <- read.csv(file = './cellchat_database/geneInfo_input_CellChatDB.csv', row.names = 1)

In [ ]:
CellChatDB <- CellChatDB.human

CellChatDB <- list()
CellChatDB$interaction <- interaction_input
CellChatDB$complex <- DB$complex
CellChatDB$cofactor <- DB$cofactor
CellChatDB$geneInfo <- geneInfo

In [ ]:
CellChatDB.use <- CellChatDB # use Secreted Signaling
# use all CellChatDB for cell-cell communication analysis
# CellChatDB.use <- CellChatDB # simply use the default CellChatDB

# set the used database in the object
cellchat_all@DB <- CellChatDB.use

cellchat_all <- setIdent(cellchat_all, ident.use = "new_clus", levels = cell.levels)

In [ ]:
cellchat_all@idents <- factor(cellchat_all@idents, levels=cell.levels)

In [ ]:
unique(cellchat_all@idents)

In [ ]:
# subset the expression data of signaling genes for saving computation cost
cellchat_all <- subsetData(cellchat_all) # This step is necessary even if using the whole database
future::plan("multiprocess", workers = 1) # do parallel
#> Warning: Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead,
#> explicitly specify either 'multisession' or 'multicore'. In the current R
#> session, 'multiprocess' equals 'multisession'.
#> Warning in supportsMulticoreAndRStudio(...): [ONE-TIME WARNING] Forked
#> processing ('multicore') is not supported when running R from RStudio
#> because it is considered unstable. For more details, how to control forked
#> processing or not, and how to silence this warning in future R sessions, see ?
#> parallelly::supportsMulticore
cellchat_all <- suppressWarnings(identifyOverExpressedGenes(cellchat_all))
cellchat_all <- suppressWarnings(identifyOverExpressedInteractions(cellchat_all))
 
# project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` in the function `computeCommunProb()` in order to use the projected data)
# cellchat <- projectData(cellchat, PPI.mouse)

In [ ]:
cellchat_all <- suppressWarnings(computeCommunProb(cellchat_all, type = "truncatedMean", trim = 0.1, 
                               distance.use = TRUE, interaction.length = 200, scale.distance = 0.1))

In [ ]:

#> truncatedMean is used for calculating the average gene expression per cell group. 
#> [1] ">>> Run CellChat on spatial imaging data using distances as constraints <<< [2022-11-12 07:49:23]"
#> The suggested minimum value of scaled distances is in [1,2], and the calculated value here is  1.30553 
#> [1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2022-11-12 08:10:42]"

# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat_all <- suppressWarnings(filterCommunication(cellchat_all, min.cells = 0))

In [ ]:
cellchat_all <- computeCommunProbPathway(cellchat_all)

In [ ]:
cellchat_all <- aggregateNet(cellchat_all)

In [ ]:
cellchat_all

In [ ]:
unique(cellchat_all@DB$interaction$pathway_name)

In [ ]:
unique(cellchat_all@LR$LRsig$pathway_name)

In [ ]:
head(cellchat_all@images)

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
groupSize <- as.numeric(table(cellchat_all@idents))
par(mfrow = c(1,2), xpd=TRUE)
netVisual_circle(cellchat_all@net$count, vertex.weight = rowSums(cellchat_all@net$count), weight.scale = T, 
                 label.edge= F, title.name = "Number of interactions",vertex.label.cex = 0.45)
netVisual_circle(cellchat_all@net$weight, vertex.weight = rowSums(cellchat_all@net$weight),
                 weight.scale = T, label.edge= F, title.name = "Interaction weights/strength",
                vertex.label.cex = 0.45)

In [ ]:
# Compute the network centrality scores
cellchat_all <- netAnalysis_computeCentrality(cellchat_all, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
saveRDS(cellchat_all, file = "cellchat_cosmx_all_replace_scrnaseq.rds")

#### cellchat figure

In [ ]:
library(CellChat)

In [ ]:
#options(repr.plot.width=7, repr.plot.height=7)
pathways.show <- c("NRG") 
# Circle plot
par(mfrow=c(1,1))
p1 <- netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
class(p1)

In [ ]:
library(svglite)

In [ ]:
#options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("WNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
options(repr.plot.width=14,repr.plot.height=14)
pathways.show <- c("ncWNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("PDGF") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
#options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("EPHB") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("NOTCH") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 0.45)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("IGF") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 0.45)

In [ ]:
pathway <- cellchat_all@LR$LRsig

In [ ]:
pathway <- cellchat_all@LR$LRsig[cellchat_all@LR$LRsig$annotation!='ECM-Receptor',]$pathway

In [ ]:
pathway <- unique(pathway)

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=30,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling='PTN')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=20,repr.plot.height=8)
netVisual_bubble(cellchat_all, signaling='IGF')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=8,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling='BMP')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=15,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling='GDF')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=15)
netVisual_bubble(cellchat_all,sources.use = 'PDGFR+FB', targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=7,repr.plot.height=7)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = c('NK'), sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = c('macrophage'), sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=2.5)
netVisual_bubble(cellchat_all, signaling=pathway,sources.use = c('macrophage'), targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = c('endothelial'), sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,sources.use = c('endothelial'), targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = 'NK', sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=4,repr.plot.height=5)
netVisual_bubble(cellchat_all,signaling=pathway, sources.use = 'PDGFR+FB', targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=6,repr.plot.height=6)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = 'PDGFR+FB')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=7,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = 'PDGFR+FB')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=7,repr.plot.height=5)
netVisual_bubble(cellchat_all, remove.isolate = TRUE,signaling=pathway,
                 sources.use = c('PDGFR+FB','macrophage','endothelial'),
                 targets.use = c('PDGFR+FB','macrophage','endothelial')
                 )
#> Comparing communications on a single object

In [ ]:
cell.levels

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=20,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling = 'WNT', targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
cellchat_all <- netAnalysis_computeCentrality(cellchat_all, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht1 <- netAnalysis_signalingRole_heatmap(cellchat_all, pattern = "outgoing",width = 14,height = 14)

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht2 <- netAnalysis_signalingRole_heatmap(cellchat_all, pattern = "incoming",width = 14,height = 14)

In [ ]:
ht1

In [ ]:
ht2

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, sources.use = c('NK'), targets.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, sources.use = c('macrophage'), targets.use = c('PDGFR+FB','CD8 T memory','CD8 T naive','CD4 T memory ','CD4 T naive','Treg','B-cell'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_pyloric, targets.use = c('macrophage'), sources.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, sources.use = c('neutrophil'), targets.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=10)
netVisual_bubble(cellchat_all, sources.use = c('PDGFR+FB'), targets.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, targets.use = c('PDGFR+FB'), sources.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
pathways.show <- c("MIF") 
# Circle plot
options(repr.plot.width=7,repr.plot.height=7)
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
pathways.show <- c("PDGF") 
# Circle plot
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 0.45)

#### cellchat figure

In [ ]:
library(CellChat)

In [ ]:
cellchat_all <- readRDS('cellchat_cosmx_all.rds')

In [ ]:
#options(repr.plot.width=7, repr.plot.height=7)
pathways.show <- c("NRG") 
# Circle plot
par(mfrow=c(1,1))
p1 <- netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
pdf("Fig7D.pdf", width=14, height=14)
p1 <- netVisual_aggregate(cellchat_all, signaling = 'NRG', layout = "circle",vertex.label.cex = 1.5)
dev.off()

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
pdf("Fig7B1.pdf", width=14, height=14)
p1 <- netVisual_aggregate(cellchat_all, signaling = 'WNT', layout = "circle",vertex.label.cex = 1.5)
dev.off()

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
pdf("Fig7B2.pdf", width=14, height=14)
p1 <- netVisual_aggregate(cellchat_all, signaling = 'ncWNT', layout = "circle",vertex.label.cex = 1.5)
dev.off()

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
pdf("Fig7F.pdf", width=14, height=14)
p1 <- netVisual_aggregate(cellchat_all, signaling = 'PDGF', layout = "circle",vertex.label.cex = 1.5)
dev.off()

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
pdf("FigS8B2.pdf", width=14, height=14)
p1 <- netVisual_aggregate(cellchat_all, signaling = 'EPHB', layout = "circle",vertex.label.cex = 1.5)
dev.off()

In [ ]:
class(p1)

In [ ]:
library(svglite)

In [ ]:
svglite(file='test.svg')
p1
dev.off()

In [ ]:
save('test.pdf',p1)

In [ ]:
#options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("WNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
options(repr.plot.width=14,repr.plot.height=14)
pathways.show <- c("ncWNT") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
#options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("PDGF") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
#options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("EPHB") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 1.5)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("NOTCH") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 0.45)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("IGF") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 0.45)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("BMP") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
pathways.show <- c("TGFb") 
# Circle plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle")

In [ ]:
pathway <- cellchat_all@LR$LRsig

In [ ]:
pathway <- cellchat_all@LR$LRsig[cellchat_all@LR$LRsig$annotation!='ECM-Receptor',]$pathway

In [ ]:
pathway <- unique(pathway)

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=30,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling='WNT')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=8,repr.plot.height=3)
netVisual_bubble(cellchat_all, signaling='ncWNT')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=20,repr.plot.height=8)
netVisual_bubble(cellchat_all, signaling='IGF')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=8,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling='BMP')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=15,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling='GDF')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=15,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling='APP')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=8,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling='TGFb')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling=pathway,sources.use = 'NK', targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=7,repr.plot.height=7)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = c('NK'), sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = c('macrophage'), sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=2.5)
netVisual_bubble(cellchat_all, signaling=pathway,sources.use = c('macrophage'), targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = c('endothelial'), sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,sources.use = c('endothelial'), targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = 'NK', sources.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=4,repr.plot.height=5)
netVisual_bubble(cellchat_all,signaling=pathway, sources.use = 'PDGFR+FB', targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=6,repr.plot.height=6)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = 'PDGFR+FB')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=7,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling=pathway,targets.use = 'PDGFR+FB')
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=7,repr.plot.height=5)
netVisual_bubble(cellchat_all, remove.isolate = TRUE,signaling=pathway,
                 sources.use = c('PDGFR+FB','macrophage','endothelial'),
                 targets.use = c('PDGFR+FB','macrophage','endothelial')
                 )
#> Comparing communications on a single object

In [ ]:
cell.levels

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=20,repr.plot.height=5)
netVisual_bubble(cellchat_all, signaling = 'WNT', targets.use = cell.levels[1:16])
#> Comparing communications on a single object

In [ ]:
cellchat_all <- netAnalysis_computeCentrality(cellchat_all, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht1 <- netAnalysis_signalingRole_heatmap(cellchat_all, pattern = "outgoing",width = 14,height = 14)

In [ ]:
# Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways
options(repr.plot.width=24,repr.plot.height=24)
ht2 <- netAnalysis_signalingRole_heatmap(cellchat_all, pattern = "incoming",width = 14,height = 14)

In [ ]:
ht1

In [ ]:
ht2

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, sources.use = c('NK'), targets.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, sources.use = c('macrophage'), targets.use = c('PDGFR+FB','CD8 T memory','CD8 T naive','CD4 T memory ','CD4 T naive','Treg','B-cell'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_pyloric, targets.use = c('macrophage'), sources.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, sources.use = c('neutrophil'), targets.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=10)
netVisual_bubble(cellchat_all, sources.use = c('PDGFR+FB'), targets.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=5,repr.plot.height=5)
netVisual_bubble(cellchat_all, targets.use = c('PDGFR+FB'), sources.use = c('GKN+F','Foveolar','Ent','TA_Ent','Ent2','LEFTY1_epi','OLFM4+Epi','Pyloric_deep','PC','Chief','ADH1+GKN1-F'),  remove.isolate = FALSE)
#> Comparing communications on a single object

In [ ]:
pathways.show <- c("MIF") 
# Circle plot
options(repr.plot.width=7,repr.plot.height=7)
par(mfrow=c(1,1))
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle")

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
pathways.show <- c("PDGF") 
# Circle plot
netVisual_aggregate(cellchat_all, signaling = pathways.show, layout = "circle",vertex.label.cex = 0.45)

# Figure作成

### create seurat file to h5ad

In [ ]:
library(Seurat)
#library(SeuratData)
library(SeuratDisk)

In [ ]:
ss <- readRDS('./cosmx_seurat_object/cosmx_all_seurat_final_before_submission.rds')

In [ ]:
ss

In [ ]:
head(ss@meta.data)

In [ ]:
SaveH5Seurat(ss, filename = 'cosmx_all_seurat_final_before_submission.h5Seurat')

In [ ]:
Convert('cosmx_all_seurat_final_before_submission.h5Seurat', dest = 'h5ad')

In [ ]:
library(Giotto)

In [ ]:
fov_join <- loadGiotto('./saveGiottoDir_final_before_submission/')

In [ ]:
smallfov <- subsetGiotto(fov_join, cell_ids = pDataDT(fov_join)[grepl('^Fundic-fov020',pDataDT(fov_join)$cell_ID)]$cell_ID)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('CHGA','APOA1','LEFTY1'),
                    # feats_color_code = c('green','yellow','black'),
                     point_size = 1,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
unique(pDataDT(smallfov)$new_clus)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('APOA1','PSCA'),
                     feats_color_code = c('white','light green'),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                     feats = list('LEFTY1','MKI67'),
                     feats_color_code = c('black','white'),
                     point_size = 5,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
smallfov2 <- subsetGiotto(fov_join, 
                         cell_ids = fov_join@cell_metadata$cell$rna$cell_ID[grepl('Fundic-fov007',fov_join@cell_metadata$cell$rna$cell_ID)])    

In [ ]:
smallfov <- subsetGiotto(fov_join, 
                         cell_ids = fov_join@cell_metadata$cell$rna$cell_ID[grepl('Fundic-fov006',fov_join@cell_metadata$cell$rna$cell_ID)])    

In [ ]:
color_set <- c('Foveolar'='#FFFFCC',
                                           'GKN+F'='#C7E9B4',
                                           'ADH1+GKN1-F'='#7FCDBB',
                                           'PG/Neck1'='#41B6C4',
                                           'Chief'='#2C7FB8',
                                           'PC'='#253494',
                                           'PG/Neck2'='#31A354',
                                           'Pyloric_deep'='#006837',
                                           'Ent'='#FEE5D9',
                                           'TA_Ent'='#FB6A4A',
                                           'Ent2'='#FCAE91',
                                           'OLFM4+Epi'='#CB181D',
                                           'Gob'='blue',
                                           'NE1'='Salmon',
                                           'NE2'='yellow',
                                           'LEFTY1_epi'='red',
                                           'B-cell'='#87CEEB',
                                           'pro_B-cell'='#90EE90',
                                           'plasma'='#DDA0DD',
                                           'T CD4 memory'='#696969',
                                           'T CD8 memory'='#556B2F',
                                           'T CD4 naive'='#696969',
                                           'T CD8 naive'='#556B2F',
                                           'Treg'='#800000',
                                           'NK'='black',
                                           'macrophage'='#F0E68C',
                                           'mast'='#F0E68C',
                                           'mDC'='#F0E68C',
                                           'monocyte'='#F0E68C',
                                           'pDC'='#F0E68C',
                                           'neutrophil'='#EE82EE',
                                           'CCL11+FB'='#4682B4',
                                           'KLF+FB'='Cyan',
                                           'PDGFR+FB'='Gray',
                                           'pericyte'='pink',
                                           'FibSmo'='#FFD700',
                                           'Stromal'='#ADFF2F',
                                           'pericyte_SM'='#FAFAD2',
                                           'endothelial'='#B22222')

In [ ]:
color_submucosa <- c('Foveolar'='#F0E68C',
                                           'GKN+F'='#F0E68C',
                                           'ADH1+GKN1-F'='darkgreen',
                                           'PG/Neck1'='#FEE5D9',
                                           'Chief'='lightgray',
                                           'PC'='#F0E68C',
                                       'PG/Neck2'='#FB6A4A',
                                            'mDC'='#FFFFCC',
                                           'macrophage'='#C7E9B4',
                                           'mast'='#7FCDBB',
                                           'monocyte'='#41B6C4',
                                           'Chief'='salmon',
                                           'pDC'='#253494',
                                           'neutrophil'='#31A354',
                                          # 'Pyloric_deep'='#006837',
                                          # 'Ent'='#FEE5D9',
                                           #'TA_Ent'='#FB6A4A',
                                          # 'Ent2'='#FCAE91',
                                           'OLFM4+Epi'='#CB181D',
                                           'Gob'='blue',
                                           'NE1'='Salmon',
                                           'NE2'='yellow',
                                           'LEFTY1_epi'='#CB181D',
                                           'B-cell'='#87CEEB',
                                           'pro_B-cell'='#90EE90',
                                           'plasma'='#DDA0DD',
                                           'T CD4 memory'='purple',
                                           'T CD8 memory'='yellow',
                                           'T CD4 naive'='blue',
                                           'T CD8 naive'='green',
                                           'Treg'='#800000',
                                           'NK'='black',
                                           #'macrophage'='#F0E68C',
                                           #'mast'='#F0E68C',
                                           #'mDC'='#F0E68C',
                                           #'monocyte'='#F0E68C',
                                           #'pDC'='#F0E68C',
                                           #'neutrophil'='#EE82EE',
                                           'CCL11+FB'='#4682B4',
                                           'KLF+FB'='Cyan',
                                           'PDGFR+FB'='Gray',
                                           'pericyte'='pink',
                                           'FibSmo'='#FFD700',
                                           'Stromal'='#ADFF2F',
                                           'pericyte_SM'='#FAFAD2',
                                           'endothelial'='red')

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov2, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('CHGA','APOA1','LEFTY1'),
                    # feats_color_code = c('green','yellow','black'),
                     point_size = 1,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_submucosa,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('CHGA','APOA1','LEFTY1'),
                    # feats_color_code = c('green','yellow','black'),
                     point_size = 1,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_submucosa,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('CHGA','APOA1','LEFTY1'),
                    # feats_color_code = c('green','yellow','black'),
                     point_size = 1,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'Tissue_spatial_region',
                     polygon_fill_as_factor = TRUE,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
smallfov <- subsetGiotto(fov_join, 
                         cell_ids = fov_join@cell_metadata$cell$rna$cell_ID[grepl('Fundic-fov013',fov_join@cell_metadata$cell$rna$cell_ID)])    

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,
                   #  feats = list('CHGA','APOA1','LEFTY1'),
                    # feats_color_code = c('green','yellow','black'),
                     point_size = 1,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     background_color = 'white',
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,background_color = 'white',
                     feats = list('FZD5','WNT5A'),
                     feats_color_code = c('green','black'),
                     point_size = 3,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     coord_fix_ratio = TRUE)

In [ ]:
# Visualize ICF expression
options(repr.plot.width=28, repr.plot.height=28)
spatInSituPlotPoints(smallfov, show_image = FALSE, polygon_alpha = 1,legend_text = 20,background_color = 'white',
                     feats = list('IGF1','IGF1R'),
                     feats_color_code = c('green','black'),
                     point_size = 3,
                     show_polygon = TRUE,
                     polygon_feat_type = 'cell',
                     polygon_color = 'white',
                     polygon_line_size = 0.1,
                     polygon_fill = 'new_clus',
                     polygon_fill_as_factor = TRUE,
                     polygon_fill_code = color_set,
                     coord_fix_ratio = TRUE)

In [ ]:
1+1

In [ ]:
# show all the significant interactions (L-R pairs) associated with certain signaling pathways
options(repr.plot.width=8,repr.plot.height=4)
netVisual_bubble(cellchat_all, signaling='IGF')
#> Comparing communications on a single object